# [1.5.3] OthelloGPT (exercises)

> **ARENA [Streamlit Page](https://arena-chapter1-transformer-interp.streamlit.app/33_🔬_[1.5.3]_OthelloGPT)**
>
> **Colab: [exercises](https://colab.research.google.com/github/callummcdougall/ARENA_3.0/blob/main/chapter1_transformer_interp/exercises/part53_othellogpt/1.5.3_OthelloGPT_exercises.ipynb?t=20250910) | [solutions](https://colab.research.google.com/github/callummcdougall/ARENA_3.0/blob/main/chapter1_transformer_interp/exercises/part53_othellogpt/1.5.3_OthelloGPT_solutions.ipynb?t=20250910)**

Please send any problems / bugs on the `#errata` channel in the [Slack group](https://join.slack.com/t/arena-uk/shared_invite/zt-3afdmdhye-Mdb3Sv~ss_V_mEaXEbkABA), and ask any questions on the dedicated channels for this chapter of material.

You can collapse each section so only the headers are visible, by clicking the arrow symbol on the left hand side of the markdown header cells.

Links to all other chapters: [(0) Fundamentals](https://arena-chapter0-fundamentals.streamlit.app/), [(1) Transformer Interpretability](https://arena-chapter1-transformer-interp.streamlit.app/), [(2) RL](https://arena-chapter2-rl.streamlit.app/).

<img src="https://raw.githubusercontent.com/info-arena/ARENA_img/main/misc/headers/header-15-3.png" width="350">

# Introduction

*Note - unless otherwise specified, first person here refers to the primary researcher, Neel Nanda.*

[Emergent World Representations](https://arxiv.org/pdf/2210.13382) is a fascinating recent ICLR Oral paper from Kenneth Li et al, summarised in [Kenneth's excellent post on the Gradient](https://thegradient.pub/othello/). They trained a model (Othello-GPT) to play legal moves in the board game Othello, by giving it random games (generated by choosing a legal next move uniformly at random) and training it to predict the next move. The headline result is that Othello-GPT learns an emergent world representation - despite never being explicitly given the state of the board, and just being tasked to predict the next move, it learns to compute the state of the board at each move. (Note that the point of Othello-GPT is to play legal moves, not good moves, though they also study a model trained to play good moves.)

They present two main pieces of evidence. They can extract the board state from the model's residual stream via non-linear probes (a two layer ReLU MLP). And they can use the probes to causally intervene and change the model's representation of the board (by using gradient descent to have the probes output the new board state) - the model now makes legal moves in the new board state even if they are not legal in the old board, and even if that board state is impossible to reach by legal play!

I've strengthened their headline result by finding that much of their more sophisticated (and thus potentially misleading) techniques can be significantly simplified. Not only does the model learn an emergent world representation, it learns a linear emergent world representation, which can be causally intervened on in a linear way! But rather than representing "this square has a black/white piece", it represents "this square has my/their piece". The model plays both black and white moves, so this is far more natural from its perspective. With this insight, the whole picture clarifies significantly, and the model becomes far more interpretable!

### The research process

You can read more about the research process in [this post here](https://www.lesswrong.com/s/nhGNHyJHbrofpPbRG/p/TAz44Lb9n9yf52pv8), which we'd strongly recommend. The exercises are structured in a different way to the chronological research process (e.g. we look at probes early on, when actually training a probe is a high-effort thing and if you were trying to get traction on a problem like this you'd want to use more basic techniques first, like logit lens / attention pattern analysis).

As you're going through these exercises, we encourage you to keep thinking about how you would approach this research process. Do any of the results / approaches seem completely out of the blue for you, and if so can you think about what the justification might have been for trying them? What would you have tried first, and why?

### How you should approach these exercises

There's a lot of setup code for analysing OthelloGPT (this is somewhat unavoidable). There are also a lot of different plots of tensors with different dimensions and different meanings, and it can be hard to keep track of. You don't have to keep track of all of this in your head, but **we strongly recommend explaining to yourself or your pair programming partner what the significance of each result is before you move on to the next section.** Sometimes exercises and questions will prompt you to do this, but you should make it a reflex!

At the end of each subsection in the first section (which mostly consists of setup), there's a recap of all the objects we've defined, why they matter, and how they can be used. There will also be a "recap of this section" at the end of each of the later sections, where we review the key results and their significance.

## The purpose / structure of these exercises

At a surface level, these exercises are designed to help you understand the OthelloGPT model and the forms of probing & analysis that were done on it. But it's also designed to make you a better interpretability researcher! As a result, most exercises will be doing a combination of:

1. Showing you some new feature/component of OthelloGPT, and
2. Teaching you how to use tools and interpret results in a broader mech interp context.

As you're going through these exercises, it's easy to get lost in the fiddly details of the techniques you're implementing or the things you're computing. Make sure you keep taking a high-level view, asking yourself what questions you're currently trying to ask and how you'll interpret the output you're getting, as well as how the tools you're currently using are helping guide you towards a better understanding of the model.

## Content & Learning Objectives

### 1️⃣ Model Setup & Linear Probes

In this section, we'll set up the model that we'll use for the rest of the exercises, and load in weights. We'll also get familiar with the datasets and objects that we'll be using for this analysis. Lastly, we'll learn about **linear probes** and how they are used.

> ##### Learning Objectives
>
> - Understand the basic structure of the Othello-GPT model
> - Become familiar with the basic tools we'll use for board visualization during these exercises
> - See how our linear probe works, and how we can change basis between "black/white piece" and "mine/their piece"
> - Intervene with the linear probe to influence model predictions

### 2️⃣ Looking for modular circuits

Here, we'll use our probe to start to analyse circuits in our model. We can apply them to our neurons' output weights to identify which neurons matter and in what way, and we can also figure out when and where information is represented in a model.

> ##### Learning Objectives
>
> - Learn how to use our linear probe across multiple layers
> - Apply activation patching at a given sequence position to test hypotheses about our model
> - Understand how a neuron can be characterized in terms of its input and output weights

### 3️⃣ Neuron Interpretability: A Deep Dive

To practice neuron interpretability, we'll take a deep dive into understanding one particular neuron - the techniques and code should transfer pretty well to any other neurons!

The spirit of this section is to practice doing various standard things that you could go and apply to another neuron in practice - I end it still being fairly confused, and with many dangling threads!

> ##### Learning Objectives
>
> - Apply **direct logit attribution** to understand how a neuron's output weights directly effect predictions
> - Use SVD-based techniques to assess how much of a neuron's input/output behaviour is captured by some subspace
> - Use techniques like **max activating datasets** and **spectrum plots**, and understand their strengths and limitations

### 4️⃣ Training a Probe

In this section, we'll look at how to actually train our linear probe. This section is less mechanistic interpretability and more standard ML techniques, but it's still important to understand how to do this if you want to do any analysis like this yourself!

> ##### Learning Objectives
>
> - Learn how to set up and train a linear probe
> - See how to train multiple probes at once, and log their performance to Weights & Biases

### ☆ Bonus

Finally, we'll take a look at some future directions which could come from this OthelloGPT analysis. We strongly recommend you follow some of these threads of research yourself, and see where they might lead!

## Setup code

In [1]:
import os
import sys
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules

chapter = "chapter1_transformer_interp"
repo = "ARENA_3.0"
branch = "main"

# Install dependencies
try:
    import transformer_lens
except:
    %pip install plum-dispatch==2.1
    %pip install einops eindex-callum jaxtyping wandb transformer_lens==2.11.0 "git+https://github.com/neelnanda-io/neel-plotly"

# Get root directory, handling 3 different cases: (1) Colab, (2) notebook not in ARENA repo, (3) notebook in ARENA repo
root = (
    "/content"
    if IN_COLAB
    else "/root"
    if repo not in os.getcwd()
    else str(next(p for p in Path.cwd().parents if p.name == repo))
)

if Path(root).exists() and not Path(f"{root}/{chapter}").exists():
    if not IN_COLAB:
        !sudo apt-get install unzip
        %pip install jupyter ipython --upgrade

    if not os.path.exists(f"{root}/{chapter}"):
        !wget -P {root} https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/{branch}.zip
        !unzip {root}/{branch}.zip '{repo}-{branch}/{chapter}/exercises/*' -d {root}
        !mv {root}/{repo}-{branch}/{chapter} {root}/{chapter}
        !rm {root}/{branch}.zip
        !rmdir {root}/{repo}-{branch}


if f"{root}/{chapter}/exercises" not in sys.path:
    sys.path.append(f"{root}/{chapter}/exercises")

os.chdir(f"{root}/{chapter}/exercises")

  Attempting uninstall: plum-dispatch
    Found existing installation: plum-dispatch 2.6.1
    Uninstalling plum-dispatch-2.6.1:
      Successfully uninstalled plum-dispatch-2.6.1
  Cloning https://github.com/neelnanda-io/neel-plotly to /tmp/pip-req-build-4xsfsv59
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/neel-plotly /tmp/pip-req-build-4xsfsv59
  Resolved https://github.com/neelnanda-io/neel-plotly to commit 6dc24b26f8dec991908479d7445dae496b3430b7
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 32.1 MB/s eta 0:00:00
  Created wheel for neel_plotly: filename=neel_plotly-0.0.0-py3-none-any.whl size=10188 sha256=4282c2a15b62b8238fa483f344ec2efd471388f40a5b3d62ae5ab3a6687e4570
  Stored in directory: /tmp/pip-ephem-wheel-cache-8svrl6vr/wh

In [2]:
import copy
import os
import sys
from dataclasses import dataclass
from functools import partial
from pathlib import Path
from typing import Callable

import einops
import numpy as np
import pandas as pd
import plotly.express as px
import torch as t
import wandb
from eindex import eindex
from jaxtyping import Bool, Float, Int
from torch import Tensor
from tqdm.notebook import tqdm
from transformer_lens import ActivationCache, HookedTransformer, HookedTransformerConfig
from transformer_lens.hook_points import HookPoint
from transformer_lens.utils import download_file_from_hf, get_act_name, to_numpy

device = t.device("mps" if t.backends.mps.is_available() else "cuda" if t.cuda.is_available() else "cpu")

# Make sure exercises are in the path
chapter = "chapter1_transformer_interp"
section = "part53_othellogpt"
root_dir = next(p for p in Path.cwd().parents if (p / chapter).exists())
exercises_dir = root_dir / chapter / "exercises"
section_dir = exercises_dir / section

import part53_othellogpt.tests as tests
import part53_othellogpt.utils as utils
from neel_plotly import scatter

t.set_grad_enabled(False)

MAIN = __name__ == "__main__"

# 1️⃣ Model Setup & Linear Probes

> ##### Learning Objectives
>
> - Understand the basic structure of the Othello-GPT model
> - Become familiar with the basic tools we'll use for board visualization during these exercises
> - See how our linear probe works, and how we can change basis between "black/white piece" and "mine/their piece"
> - Intervene with the linear probe to influence model predictions

For those unfamiliar, Othello is a board game analogous to chess or go, with two players, black and white, see the rules outlined in the figure below. I found [playing the AI on eOthello](https://www.eothello.com/) helpful for building intuition. A single move can change the colour of pieces far away (so long as there's a continuous vertical, horizontal or diagonal line), which means that calculating board state is actually pretty hard! (to my eyes much harder than in chess)

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/nmxzr2zsjNtjaHh7x/eoi5u5fodorjswnmbqk7" width="700">

## Background - Othello, and OthelloGPT

But despite the model just needing to predict the next move, it spontaneously learned to compute the full board state at each move - a fascinating result. A pretty hot question right now is whether LLMs are just bundles of statistical correlations or have some real understanding and computation! This gives suggestive evidence that simple objectives to predict the next token can create rich emergent structure (at least in the toy setting of Othello). Rather than just learning surface level statistics about the distribution of moves, it learned to model the underlying process that generated that data. In my opinion, it's already pretty obvious that transformers can do something more than statistical correlations and pattern matching, see eg [induction heads](https://transformer-circuits.pub/2022/in-context-learning-and-induction-heads/index.html), but it's great to have clearer evidence of fully-fledged world models!

For context on my investigation, it's worth analysing exactly the two pieces of evidence they had for the emergent world representation, the probes and the causal interventions, and their strengths and weaknesses.

### Probes

The probes give suggestive, but far from conclusive evidence. When training a probe to extract some feature from a model, it's easy to trick yourself. It's crucial to track whether the probe is just reading out the feature, or actually computing the feature itself, and reading out much simpler features from the model. In the extreme case, you could attach a much more powerful model as your "probe", and have it just extract the input moves, and then compute the board state from scratch! They found that linear probes did not work to recover board state (with an error rate of 20.4%): (ie, projecting the residual stream onto some 3 learned directions for each square, corresponding to empty, black and white logits). While the simplest non-linear probes (a two layer MLP with a single hidden ReLU layer) worked extremely well (an error rate of 1.7%). Further (as described in their table 2, screenshot below), these non-linear probes did not work on a randomly initialised network, and worked better on some layers than others, suggesting they were learning something real from the model.

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/nmxzr2zsjNtjaHh7x/b5rxfns1wxvsuomh1d79" width="550">

### Causal interventions

Probes on their own can mislead, and don't necessarily tell us that the model uses this representation - the probe could be extracting some vestigial features or a side effect of some more useful computation, and give a misleading picture of how the model computes the solution. But their causal interventions make this much more compelling evidence. They intervene by a fairly convoluted process (detailed in the figure below, though you don't need to understand the details), which boils down to choosing a new board state, and applying gradient descent to the model's residual stream such that our probe thinks the model's residual stream represents the new board state. I have an immediate skepticism of any complex technique like this: when applying a powerful method like gradient descent it's so easy to wildly diverge from what the models original functioning is like! But the fact that the model could do the non-trivial computation of converting an edited board state into a legal move post-edit is a very impressive result! I consider it very strong evidence both that the probe has discovered something real, and that the representation found by the probe is causally linked to the model's actual computation!

## Naive Implications for Mechanistic Interpretability

I was very interested in this paper, because it simultaneously had the fascinating finding of an emergent world model (and I'm also generally into any good interp paper), yet something felt off. The techniques used here seemed "too" powerful. The results were strong enough that something here seemed clearly real, but my intuition is that if you've really understood a model's internals, you should just be able to understand and manipulate it with far simpler techniques, like linear probes and interventions, and it's easy to be misled by more powerful techniques.

In particular, my best guess about model internals is that the networks form [**decomposable, linear representations**](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=KiDMIteKCEXt_EkR2sZCOfbG): that the model computes a bunch of useful features, and represents these as directions in activation space. See [Toy Models of Superposition](https://transformer-circuits.pub/2022/toy_model/index.html#motivation) for some excellent exposition on this. This is decomposable because each feature can vary independently (from the perspective of the model - on the data distribution they're likely dependent), and linear because we can extract a feature by projecting onto that feature's direction (if the features are orthogonal - if we have something like [superposition](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=3br1psLRIjQCOv2T4RN3V6F2) it's messier). This is a natural way for models to work - they're fundamentally a series of matrix multiplications with some non-linearities stuck in convenient places, and a decomposable, linear representation allows it to extract any combination of features with a linear map!

Under this framework, if a feature can be found by a linear probe then the model has already computed it, and if that feature is used in a circuit downstream, we should be able to causally intervene with a linear intervention, just changing the coordinate along that feature's direction. So the fascinating finding that linear probes do not work, but non-linear probes do, suggests that either the model has a fundamentally non-linear representation of features (which it is capable of using directly for downstream computation!), or there's a linear representation of simpler and more natural features, from which the probe computes board state. My prior was on a linear representation of simpler features, but the causal intervention findings felt like moderate evidence for the non-linear representation. And the non-linear representation hypothesis would be a big deal if true! If you want to reverse-engineer a model, you need to have a crisp picture of how its computation maps onto activations and weights, and this would break a lot of my beliefs about how this correspondance works! Further, linear representations are just really *convenient* to reverse-engineer, and this would make me notably more pessimistic about mechanistic interpretability working.

## Model Setup & Visualization

The best way to become less confused about a mysterious model behaviour is to mechanistically analyse it. To zoom in on whatever features and circuits we can find, build our understanding from the bottom up, and use this to form grounded beliefs about what's actually going on.

To get started, let's load in the model that we'll be using for this chapter. A few important technical details:

- Othello games have 60 moves (the board has size 8x8, and the middle 4 squares are all occupied, so by 60 moves all squares are occupied by some color). The model's context length is 59, because it's learned to take in `moves[0:59]` and predict `moves[1:60]`.
- The dataset the model was trained on just consists of randomly sampled legal moves. A move in Othello is legal if and only if it captures the opponents' pieces in a horizontal, vertical or diagonal line - in which case it flips all those pieces to its own color. This means **predicting next legal moves is a nontrivial task (one might guess it requires the mdoel to track captures and board states over time)**
    - Because the model was trained on cross entropy loss against the actual next moves, the model's distribution converges to uniform over all next legal moves - can you see why?

<details>
<summary>Proof - why the model learns a uniform distribution over next legal moves</summary>

The cross entropy loss is defined as $H(p, q) = -\sum_x p(x) \log q(x)$, which by [Gibbs' inequality](https://en.wikipedia.org/wiki/Gibbs%27_inequality) is minimized when $p(x) = q(x)$. So we minimize loss when the model's distribution $q$ matches the underlying data distribution $p$, which is uniform over next legal moves.

Note we can also prove this using Jensen's inequality, since $\log q(x)$ is a strictly concave function. Alternatively, we can express the cross entropy loss as the sum of the underlying data distribution's entropy and the KL divergence between the model's distribution and the underlying distribution:

$$
H(p, q) = -\sum_x p(x) \log q(x) = \sum_x p(x) \log \frac{p(x)}{q(x)} - \sum_x p(x) \log p(x) = D_{\mathrm{KL}}(p \| q) + H(p)
$$

so this statement becomes equivalent to the statement that $D_{\mathrm{KL}}(p \| q)$ is always non-negative and is only zero when $p(x) = q(x)$ (which can be proved in similar ways).

</details>

- The vocabulary size is 61, because we allow any of the 8x8 - 4 = 60 unoccupied squares to be played, plus the pass move. The vocab is ordered `pass, A0, A1, ..., H7`. Note that we'll be filtering out games where a `pass` move was played, so we don't need to worry about this.
- We'll refer to squares in 3 different ways:
    1. **label** - this is the string representation, i.e. `"pass"`, `"A0"`, `"A1"`, ..., `"H7"`.
    2. **token id**, or **id** - this is the token ID in the model's vocab, i.e. `1` for A0, ..., `60` for H7. We skip `0` which is the token id for `pass`, and we skip the 4 middle squares since they're always occupied and so there are no moves in or predictions for these squares.
    3. **square index**, or **square** - this is the zero-indexed value of the square in the size-64 board, i.e. `0` for A0, `1` for A1, ..., `63` for H7.
- Black plays first in Othello, and so (in games with no passes) White plays last. Since we don't predict the very first move, this means the model's predictions are for (white 1, black 2, white 2, ..., white 29, black 30, white 30).


<!-- Here are some more files, which we won't be using directly in these exercises, but you might still like to have a look at:

* `tl_probing.py` is my probe training file. But it was used to train a second probe, linear_probe_L4_blank_vs_color_v1.pth . This probe actually didn't work very well for analysing the model (despite getting great accuracy) and I don't know why - it was trained on layer 4, to do a binary classification on blank vs not blank, and on my color vs their color *conditional *on not being blank (ie not evaluated if blank). For some reason, the "this cell is my color" direction has a significant dot product with the "is blank" direction, and this makes it much worse for e.g. interpreting neurons. I don't know why!
* `tl_scratch.py` is where I did some initial exploration, including activation patching between different final moves.
* `tl_exploration.py` is where I did my most recent exploration, verifying that the probe works, doing probe interventions (CTRL F for `newly_legal`) and using the probe to interpret neurons. -->

Run the code below, to load in the model:

In [3]:
cfg = HookedTransformerConfig(
    n_layers=8,
    d_model=512,
    d_head=64,
    n_heads=8,
    d_mlp=2048,
    d_vocab=61,
    n_ctx=59,
    act_fn="gelu",
    normalization_type="LNPre",
    device=device,
)
model = HookedTransformer(cfg)

state_dict_synthetic = download_file_from_hf("NeelNanda/Othello-GPT-Transformer-Lens", "synthetic_model.pth")
# state_dict_championship = download_file_from_hf("NeelNanda/Othello-GPT-Transformer-Lens", "championship_model.pth")

model.load_state_dict(state_dict_synthetic)

./synthetic_model.pth:   0%|          | 0.00/101M [00:00<?, ?B/s]

<All keys matched successfully>

Next, run the code below to check that the model is working correctly. Our `sample_input` is a sequence of 10 moves (the first 10 moves of the game, starting with black's 1st move and ending in white's 5th move).

In [4]:
# An example input: 10 moves in a game
sample_input = t.tensor([[20, 19, 18, 10, 2, 1, 27, 3, 41, 42]]).to(device)

logits = model(sample_input)
logprobs = logits.log_softmax(-1)

assert logprobs.shape == (1, 10, 61)  # shape is [batch, seq_len, d_vocab]
assert logprobs[0, 0].topk(3).indices.tolist() == [
    21,
    33,
    19,
]  # these are the 3 legal moves, as we'll soon show

Note that we can convert each of our logprob vectors (which have shape `(61,)`) into a board state tensor of shape `(8, 8)` (we put very negative values in the 4 middle squares) via some clever indexing. Then, we can plot the results using a helper function `utils.plot_board_values` we've written for you. Since you'll be using this function a lot in subsequent exercises, we'll go through the important arguments here:

- `state`: a tensor of shape `(*N, 8, 8)`, where `*N` is any number of batch dimensions. For each `N` tensors, we'll visualize a single 8x8 grid. If the shape is `(8, 8)` then we'll plot a single board.
- `board_titles`: if plotting multiple boards (i.e. `N > 1`) then we can use this argument to label each of them.
- `boards_per_row`: if plotting multiple boards, we can set this many boards per row.
- `text`: an optional list of strings with the same shape as `state`, which we can use to annotate the boards. Also fine for it to be broadcastable to the shape of `state` (e.g. it can be `(8, 8)` even if `state` is 3D).
- `kwargs`: other arguments get passed into `px.imshow` (e.g. `title`, `width` and `height`)

Understanding exactly how this function works isn't important, since you can always copy and paste code from one of the following examples if you forget how to use it!

In [5]:
MIDDLE_SQUARES = [27, 28, 35, 36]
ALL_SQUARES = [i for i in range(64) if i not in MIDDLE_SQUARES]

logprobs_board = t.full(size=(8, 8), fill_value=-13.0, device=device)
logprobs_board.flatten()[ALL_SQUARES] = logprobs[0, 0, 1:]  # the [1:] is to filter out logits for the "pass" move

utils.plot_board_values(logprobs_board, title="Example Log Probs", width=500)

<details>
<summary>Aside - how does this tensor indexing magic work?</summary>

`temp_board_state` is an array of shape `(8, 8)`. When we use `.flatten()`, this returns a **view** (i.e. same underlying data) with shape `(64,)`. When we index it by `ALL_SQUARES` (a list of 60 indices, which is all the indices excluding the "middle squares"), this also returns a view (still the same data). We can then set those 60 elements to be the model's log probs. This will change the values in the original tensor, without changing that tensor's shape.
</details>

We can use the `text` argument to annotate all the legal squares with their token IDs, which might be easier in some cases. Here's an example (you might want to reuse this code in some later exercises):

In [6]:
TOKEN_IDS_2D = np.array([str(i) if i in ALL_SQUARES else "" for i in range(64)]).reshape(8, 8)
BOARD_LABELS_2D = np.array(["ABCDEFGH"[i // 8] + f"{i % 8}" for i in range(64)]).reshape(8, 8)

print(TOKEN_IDS_2D)
print(BOARD_LABELS_2D)

utils.plot_board_values(
    t.stack([logprobs_board, logprobs_board]),  # shape (2, 8, 8)
    title="Example Log Probs (with annotated token IDs)",
    width=800,
    text=np.stack([TOKEN_IDS_2D, BOARD_LABELS_2D]),  # shape (2, 8, 8)
    board_titles=["Labelled by token ID", "Labelled by board label"],
)

[['0' '1' '2' '3' '4' '5' '6' '7']
 ['8' '9' '10' '11' '12' '13' '14' '15']
 ['16' '17' '18' '19' '20' '21' '22' '23']
 ['24' '25' '26' '' '' '29' '30' '31']
 ['32' '33' '34' '' '' '37' '38' '39']
 ['40' '41' '42' '43' '44' '45' '46' '47']
 ['48' '49' '50' '51' '52' '53' '54' '55']
 ['56' '57' '58' '59' '60' '61' '62' '63']]
[['A0' 'A1' 'A2' 'A3' 'A4' 'A5' 'A6' 'A7']
 ['B0' 'B1' 'B2' 'B3' 'B4' 'B5' 'B6' 'B7']
 ['C0' 'C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'C7']
 ['D0' 'D1' 'D2' 'D3' 'D4' 'D5' 'D6' 'D7']
 ['E0' 'E1' 'E2' 'E3' 'E4' 'E5' 'E6' 'E7']
 ['F0' 'F1' 'F2' 'F3' 'F4' 'F5' 'F6' 'F7']
 ['G0' 'G1' 'G2' 'G3' 'G4' 'G5' 'G6' 'G7']
 ['H0' 'H1' 'H2' 'H3' 'H4' 'H5' 'H6' 'H7']]


We can also use this function to plot multiple board states at once, exactly the same way. Again pay attention to the indexing, because understanding how this works will be useful going forwards!

In [7]:
logprobs_multi_board = t.full(size=(10, 8, 8), fill_value=-13.0, device=device)
logprobs_multi_board.flatten(1, -1)[:, ALL_SQUARES] = logprobs[0, :, 1:]  # we now do all 10 moves at once

utils.plot_board_values(
    logprobs_multi_board,
    title="Example Log Probs",
    width=1000,
    boards_per_row=5,
    board_titles=[f"Logprobs after move {i}" for i in range(1, 11)],
)

Let's use the same function to plot the first 10 board states, and check that the predictions shown above make sense given the board state at that time. We'll use a helper method `OthelloBoardState` to track the board state after each move.

In [8]:
board_states = t.zeros((10, 8, 8), dtype=t.int32)
legal_moves = t.zeros((10, 8, 8), dtype=t.int32)

board = utils.OthelloBoardState()
for i, token_id in enumerate(sample_input.squeeze()):
    # board.umpire takes a square index (i.e. from 0 to 63) and makes a move on the board
    board.umpire(utils.id_to_square(token_id))

    # board.state gives us the 8x8 numpy array of 0 (blank), -1 (black), 1 (white)
    board_states[i] = t.from_numpy(board.state)

    # board.get_valid_moves() gives us a list of the indices of squares that are legal to play next
    legal_moves[i].flatten()[board.get_valid_moves()] = 1

# Turn `legal_moves` into strings, with "o" where the move is legal and empty string where illegal
legal_moves_annotation = np.where(to_numpy(legal_moves), "o", "").tolist()

utils.plot_board_values(
    board_states,
    title="Board states",
    width=1000,
    boards_per_row=5,
    board_titles=[f"State after move {i}" for i in range(1, 11)],
    text=legal_moves_annotation,
)

In this plot you should see that each board state evolves from the previous one via a single move that captures a set of pieces from the opposing color (e.g. in the first move black plays `C3` capturing white at `D3`, and in the second move white plays `C2` and captures back black diagonally at `D3`). You should also see that the annotated legal moves match the moves predicted by the model.

## Data

Let's now load some data for OthelloGPT. We'll load data in in `id` format (i.e. 1 to 60 inclusive, since our vocab is `range(0, 61)` and we're filtering out games with pass moves) and `int` format (i.e. 0 to 63 inclusive, since the games contain moves from `A0` to `H7`).

In [9]:
board_seqs_id = t.from_numpy(np.load(section_dir / "board_seqs_id_small.npy")).long()
board_seqs_square = t.from_numpy(np.load(section_dir / "board_seqs_square_small.npy")).long()

print(f"board_seqs_id: shape {tuple(board_seqs_id.shape)}, range: {board_seqs_id.min()} to {board_seqs_id.max()}")
print(
    f"board_seqs_square: shape {tuple(board_seqs_square.shape)}, range: {board_seqs_square.min()} to {board_seqs_square.max()}"
)

board_seqs_id: shape (100000, 60), range: 1 to 60
board_seqs_square: shape (100000, 60), range: 0 to 63


Note - you can access a larger dataset at the [GitHub readme](https://github.com/likenneth/othello_world), in the "Training Othello-GPT" section. There are links to download datasets from Google Drive (both synthetic and championship games). You can store them in your `data` folder, after you've cloned the repo using the code above.

## Making some utilities

At this point, we'll stop and get some aggregate data that will be useful later - a tensor of valid moves, of board states, and a cache of all model activations across 50 games (in practice, you want as much as will comfortably fit into GPU memory). It's really convenient to have the ability to quickly run an experiment across a bunch of games! And one of the great things about small models on algorithmic tasks is that you just can do stuff like this.

Let's call these the **focus games**.

In [10]:
def get_board_states_and_legal_moves(
    games_square: Int[Tensor, "n_games n_moves"],
) -> tuple[
    Int[Tensor, "n_games n_moves rows cols"],
    Int[Tensor, "n_games n_moves rows cols"],
    list,
]:
    """
    Returns the following:
        states:                 (n_games, n_moves, 8, 8): tensor of board states after each move
        legal_moves:            (n_games, n_moves, 8, 8): tensor of 1s for legal moves, 0s for
                                    illegal moves
        legal_moves_annotation: (n_games, n_moves, 8, 8): list containing strings of "o" for legal
                                    moves (for plotting)
    """
    # Create tensors to store the board state & legal moves
    n_games, n_moves = games_square.shape
    states = t.zeros((n_games, 60, 8, 8), dtype=t.int32)
    legal_moves = t.zeros((n_games, 60, 8, 8), dtype=t.int32)

    # Loop over each game, populating state & legal moves tensors after each move
    for n in range(n_games):
        board = utils.OthelloBoardState()
        for i in range(n_moves):
            board.umpire(games_square[n, i].item())
            states[n, i] = t.from_numpy(board.state)
            legal_moves[n, i].flatten()[board.get_valid_moves()] = 1

    # Convert legal moves to annotation
    legal_moves_annotation = np.where(to_numpy(legal_moves), "o", "").tolist()

    return states, legal_moves, legal_moves_annotation


num_games = 50

focus_games_id = board_seqs_id[:num_games]  # shape [50, 60]
focus_games_square = board_seqs_square[:num_games]  # shape [50, 60]
focus_states, focus_legal_moves, focus_legal_moves_annotation = get_board_states_and_legal_moves(focus_games_square)

print("focus states:", focus_states.shape)
print("focus_legal_moves", tuple(focus_legal_moves.shape))

# Plot the first 10 moves of the first game
utils.plot_board_values(
    focus_states[0, :10],
    title="Board states",
    width=1000,
    boards_per_row=5,
    board_titles=[f"Move {i}, {'white' if i % 2 == 1 else 'black'} to play" for i in range(1, 11)],
    text=np.where(to_numpy(focus_legal_moves[0, :10]), "o", "").tolist(),
)

focus states: torch.Size([50, 60, 8, 8])
focus_legal_moves (50, 60, 8, 8)


Let's also cache all the model's activations and logits for these focus games.

In [11]:
focus_logits, focus_cache = model.run_with_cache(focus_games_id[:, :-1].to(device))

print(focus_logits.shape)  # shape [num_games=50, n_ctx=59, d_vocab=61]

torch.Size([50, 59, 61])


> #### Recap of the useful objects we've defined
>
> We have the following:
>
> Models
> - `model` is an 8-layer autoregressive transformer, trained to predict legal Othello moves. Its vocab is `range(0, 61)` where 0 = "pass" and the other numbers represent the 60 possible moves, excluding the 4 middle squares
>
> All data
> - `board_seqs_id`, shape `(100k, 60)` contains the moves from all 100k games (as token ids)
> - `board_seqs_square`, shape `(100k, 60)` contains the moves from all 100k games (as ints)
>
> Focus games data
> - `focus_games_id`, shape `(50, 60)` contains the moves from 50 games (as token ids)
> - `focus_games_square`, shape `(50, 60)` contains the moves from 50 games (as ints)
> - `focus_states`, shape `(50, 60, 8, 8)` contains the board states after each move (0 = empty, 1 = black, -1 = white)
> - `focus_legal_moves`, shape `(50, 60, 8, 8)` contains a 1 for each legal move, and 0 for each illegal move
> - `focus_logits`, shape `(50, 59, 61)` contains model's output logits on focus games - `59=model.cfg.n_ctx` because we don't include the final move in our forward pass, and `61=model.cfg.d_vocab` contains the "pass" move and all 60 playable squares

## What is a probe?

From the [MI Dynalist notes](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#q=probe):

> **[Probing](https://arxiv.org/pdf/1610.01644.pdf)** is a technique for identifying directions in network activation space that correspond to a concept/feature.
>
> In spirit, you give the network a bunch of inputs with that feature, and a bunch without it. You train a linear map on a specific activation (eg the output of layer 5) which distinguishes these two sets, giving a 1D linear map (a **probe**), corresponding to a direction in activation space, which likely corresponds to that feature.

Probes can be a very valuable tool to help us better understand the concepts represented in our model. However, there are two big caveats to keep in mind:

1. Probes give us a direction, but they don't give us a causal story about how that direction got into the model in the first place, or how the model is using that direction.
2. Probes (especially nonlinear probes) can be hiding a lot of computation under their surface.

In the original paper analysing Othello, the authors used nonlinear probing to find important directions. This went against a fundamental intuition - that models fundamentally store things in linear ways, and so we should be able to access them with linear probes. In these exercises, we'll be using linear probes.

## Using the probe

The training of this probe was kind of a mess, and I'd do a bunch of things differently if doing it again.

There were 3 different probe modes:

- `full_linear_probe[0].shape = (d_model, 8, 8, 3)` was trained on black to play, i.e. odd moves. The classes are `[empty, white, black]`.
- `full_linear_probe[1].shape = (d_model, 8, 8, 3)` was trained on white to play, i.e. even moves. The classes are `[empty, white, black]`.
- `full_linear_probe[2].shape = (d_model, 8, 8, 3)` was trained on all moves.

For example, we could take `full_linear_probe[0]` and take the inner product with the residual stream to get a tensor of shape `(8, 8, 3)` representing `8x8=64` separate predictions for what each of the 64 board squares contains (i.e. you'd softmax this tensor over the last dimension to get probabilities).

In [12]:
full_linear_probe = t.load(section_dir / "main_linear_probe.pth", map_location=str(device), weights_only=True)

print(full_linear_probe.shape)

# Define indices along `full_linear_probe.shape[0]`, i.e. the different probe modes
black_to_play, white_to_play, _ = (0, 1, 2)
# Define indices along `full_linear_probe.shape[-1]`, i.e. the different classifications for each mode
empty, white, black = (0, 1, 2)

torch.Size([3, 512, 8, 8, 3])


### Exercise - calculate probe cosine similarities

> ```yaml
> Difficulty: 🔴🔴🔴⚪⚪
> Importance: 🔵🔵🔵⚪⚪
>
> You should spend up to 10-20 minutes on this exercise.
> ```

We won't be using `full_linear_probe[2]` much, since it doesn't work very well. We'll be focusing on the first two modes.

The key result that was found in this investigation is that **the probe learns directions in terms of "theirs vs mine" rather than "black vs white"**. In this case, you'd expect the probes for odd and even moves to be approximately opposite to each other (since "theirs vs mine" has the opposite meaning in odd vs even moves). [Here's a plot](https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/nmxzr2zsjNtjaHh7x/cuxy4pf353wazoq5emmn) visualizing that we do in fact get this - it shows the cosine similarity of the "black minus white" directions across each 64 squares in each of the probe modes. The off diagonal stripe with values close to negative 1 indicates that the "black minus white" direction for any given square is approximately antipodal when taken at odd vs even modes (marked with `(O)` and `(E)` in the reference plot).

Try to replicate this plot, using just the `full_linear_probe` that's been defined for you above. Remember, it has shape `(modes=3, d_model=512, rows=8, cols=8, options=3)`: the first 2 modes are black to play / white to play, and the three options are empty / white / black. We've given you the code to create the plot from the `cosine_similarities` tensor (which should have shape `(64*2, 64*2)`), all you need to do is create the tensor.

In [13]:
# YOUR CODE HERE - define `cosine_similarities`, then run the cell to create the plot
d_model = model.cfg.d_model

black_v_white_odd = (full_linear_probe[black_to_play,:,:,:,black] - full_linear_probe[black_to_play,:,:,:,white]).view(d_model,-1) #[d_model, 8*8]
black_v_white_even = (full_linear_probe[white_to_play,:,:,:,black] - full_linear_probe[white_to_play,:,:,:,white]).view(d_model,-1) #[d_model, 8*8]

# divide by norm across d_model dim
black_v_white_odd = black_v_white_odd / black_v_white_odd.norm(dim=0)
black_v_white_even = black_v_white_even / black_v_white_even.norm(dim=0)

new_dirs = t.cat([black_v_white_odd, black_v_white_even], dim=1) #[d_model, 2*8*8]

cosine_similarities = new_dirs.T @ new_dirs

fig = px.imshow(
    to_numpy(cosine_similarities),
    title="Cosine Sim of B-W Linear Probe Directions by Cell",
    x=[f"{label} (O)" for label in BOARD_LABELS_2D.flatten()] + [f"{label} (E)" for label in BOARD_LABELS_2D.flatten()],
    y=[f"{label} (O)" for label in BOARD_LABELS_2D.flatten()] + [f"{label} (E)" for label in BOARD_LABELS_2D.flatten()],
    width=900,
    height=800,
    color_continuous_scale="RdBu",
    color_continuous_midpoint=0.0,
)
fig.show()

<details><summary>Solution</summary>

```python
# Get the "black vs white" probe directions for odd & even moves respectively
black_vs_white_dir_odd_moves = (
    full_linear_probe[black_to_play, :, :, :, black] - full_linear_probe[black_to_play, :, :, :, white]
)
black_vs_white_dir_even_moves = (
    full_linear_probe[white_to_play, :, :, :, black] - full_linear_probe[white_to_play, :, :, :, white]
)

# Flatten over (rows, cols) then concatenate them over this dimension
all_dirs = t.stack([black_vs_white_dir_odd_moves, black_vs_white_dir_even_moves])
all_dirs = einops.rearrange(all_dirs, "parity d_model rows cols -> d_model (parity rows cols)")

# Compute cosine similarities
all_dirs_normed = all_dirs / all_dirs.norm(dim=0, keepdim=True)
cosine_similarities = einops.einsum(
    all_dirs_normed,
    all_dirs_normed,
    "d_model mode_row_col_1, d_model mode_row_col_2 -> mode_row_col_1 mode_row_col_2",
)
```
</details>

### Changing probe basis

Now we've established that the probe directions are very similar, let's just average them to create our probe in a new basis: the "theirs vs mine" basis, not the "black vs white" basis. For example, our new probe's "theirs" direction will be the average of the white direction when black is to play, and the black direction when white is to play.

In [14]:
linear_probe = t.stack(
    [
        # "Empty" direction = average of empty direction across probe modes
        full_linear_probe[[black_to_play, white_to_play], ..., [empty, empty]].mean(0),
        # "Theirs" direction = average of {x to play, classification != x} across probe modes
        full_linear_probe[[black_to_play, white_to_play], ..., [white, black]].mean(0),
        # "Mine" direction = average of {x to play, classification == x} across probe modes
        full_linear_probe[[black_to_play, white_to_play], ..., [black, white]].mean(0),
    ],
    dim=-1,
)

Let's test out our new probe, by applying it to move 29 in game 0 from our focus games. This is an odd move, so black is to play.

In [15]:
def plot_probe_outputs(
    cache: ActivationCache,
    linear_probe: Tensor,
    layer: int,
    game_index: int,
    move: int,
    title: str = "Probe outputs",
):
    residual_stream = cache["resid_post", layer][game_index, move]
    probe_out = einops.einsum(residual_stream, linear_probe, "d_model, d_model row col options -> options row col")

    utils.plot_board_values(
        probe_out.softmax(dim=0),
        title=title,
        width=900,
        height=400,
        board_titles=["P(Empty)", "P(Their's)", "P(Mine)"],
        # text=BOARD_LABELS_2D,
    )


layer = 6
game_index = 0
move = 29

utils.plot_board_values(
    focus_states[game_index, move],
    title="Focus game states",
    width=400,
    height=400,
    text=focus_legal_moves_annotation[game_index][move],
)

plot_probe_outputs(
    focus_cache,
    linear_probe,
    layer,
    game_index,
    move,
    title="Probe outputs after move 29 (black to play)",
)

Moving back to layer 3, it seems the model already has a pretty good board state representation by this point, but it's missing a few things (most notably it thinks C5 and especially C6 are white when they're actually black). My guess is that the board state calculation circuits haven't quite finished and are doing some iterative reasoning - if those cells have been taken several times, maybe it needs a layer to track the next earliest time it was taken? I don't know, and figuring this out would be a great starter project if you want to explore!

In [16]:
layer = 3
game_index = 0
move = 29

plot_probe_outputs(
    focus_cache,
    linear_probe,
    layer,
    game_index,
    move,
    title="Probe outputs (layer 4) after move 29 (black to play)",
)

Now let's take one step forward - we should see that the representations totally flip. This is indeed what we find.

In [17]:
layer = 4
game_index = 0
move = 30

utils.plot_board_values(
    focus_states[game_index, move],
    text=focus_legal_moves_annotation[game_index][move],
    title="Focus game states",
    width=400,
    height=400,
)
plot_probe_outputs(
    focus_cache,
    linear_probe,
    layer,
    game_index,
    move,
    title="Probe outputs (layer 4) after move 30 (white to play)",
)

Notice that the model gets the corner wrong in this case (incorrectly thinking that the corner is white rather than empty) - it's not a big deal, but interesting!

Can you think of a reason why corners might be treated differently in this model?

<details>
<summary>Hint</summary>

One possible reason is to do with the rules of Othello, and how the corners have a special significance. What happens to a piece once it's placed in the corner?

</details>

<details>
<summary>One possible reason</summary>

A fact about Othello is that a piece in the corners can never be flanked and thus will never change colour once placed - perhaps the model has decided to cut corners and have a different and less symmetric circuit for these?

Trying to locate this circuit might be a fun bonus exercise!
</details>

### Computing accuracy

Hopefully I've convinced you anecdotally that a linear probe works. But to be more rigorous, let's check accuracy across our 50 games.

In [18]:
# Create a tensor of "their vs mine" board states (by flipping even parities of the "focus_states" tensor)
focus_states_theirs_vs_mine = focus_states * (-1 + 2 * (t.arange(focus_states.shape[1]) % 2))[None, :, None, None]

# Convert values (0: empty, 1: theirs, -1: mine) to (0: empty, 1: theirs, 2: mine)
focus_states_theirs_vs_mine[focus_states_theirs_vs_mine == 1] = 2
focus_states_theirs_vs_mine[focus_states_theirs_vs_mine == -1] = 1

# Get probe values at layer 6, and compute the probe predictions
probe_out = einops.einsum(
    focus_cache["resid_post", 6],
    linear_probe,
    "game move d_model, d_model row col options -> game move row col options",
)
probe_predictions = probe_out.argmax(dim=-1)

# Get accuracy at odd, even & all moves (average over games & moves)
correct_middle_odd_answers = (probe_predictions.cpu() == focus_states_theirs_vs_mine[:, :-1])[:, 5:-5:2]
accuracies_odd = einops.reduce(correct_middle_odd_answers.float(), "game move row col -> row col", "mean")

correct_middle_even_answers = (probe_predictions.cpu() == focus_states_theirs_vs_mine[:, :-1])[:, 6:-5:2]
accuracies_even = einops.reduce(correct_middle_even_answers.float(), "game move row col -> row col", "mean")

correct_middle_answers = (probe_predictions.cpu() == focus_states_theirs_vs_mine[:, :-1])[:, 5:-5]
accuracies = einops.reduce(correct_middle_answers.float(), "game move row col -> row col", "mean")

# Plot accuracies
utils.plot_board_values(
    1 - t.stack([accuracies_odd, accuracies_even, accuracies], dim=0),
    title="Average Error Rate of Linear Probe",
    width=1000,
    height=400,
    board_titles=["Black to play", "White to play", "All moves"],
    zmax=0.25,
    zmin=-0.25,
)

Note that we can see the probe is worse near corners, as we anecdotally observed.

## Intervening with the probe

One of the really exciting consequences of a linear probe is that it gives us a set of interpretable directions in the residual stream! And with this, we can not only interpret the model's representations, but we can also intervene in the model's reasoning. This is a good proof of concept that if you can *really* understand a model, you can get precise and detailed control over its behaviour.

The first step is to convert our probe to meaningful directions. Each square's probe has 3 vectors, but the logits go into a softmax, which is translation invariant, so this only has two degrees of freedom. A natural-ish way to convert it into two vectors is taking `blank - (mine + theirs)/2` giving a "is this cell empty or not" direction and `mine - theirs` giving a "conditional on being blank, is this my colour vs their's" direction.

<details>
<summary>Help - I'm confused by this.</summary>

If you've done the indirect object identification exercises, this is similar to looking at the `"John" - "Mary"` direction in the logit output - i.e. we take the difference between two logits, and this gets us the log-likelihood ratio between these two options.

It's slightly less principled when we're dealing with more than two different logits, because the nonlinearities get messy. However, using `blank - (mine + theirs)/2` is still a pretty reasonable metric:

* It's symmetric in `mine` and `theirs`,
* It's translation invariant (i.e. you could add a constant `c` onto all of `blank`, `mine` and `theirs` and it wouldn't change),
* If you increase `blank` by some amount `c` and keep the other two the same, then this metric also increases by `c`.

The `mine - theirs` direction is more principled.

</details>

Having a single meaningful direction is important, because it allows us to interpret a feature or intervene on it. The original three directions has one degree of freedom, so each direction is arbitrary on its own.

### Exercise - define your probe directions

> ```yaml
> Difficulty: 🔴🔴⚪⚪⚪
> Importance: 🔵🔵🔵🔵⚪
>
> You should spend up to 5-10 minutes on this exercise.
>
> The exercise isn't conceptually hard, understanding what the objects in question represent is the important part.
> ```

Define the tensors `blank_probe` and `my_prob`, which point in the two directions given above.

<details>
<summary>Help - I'm confused by exactly how to take the linear combination here.</summary>

Your `linear_probe` tensor has shape `[d_model, row, col, options]`, where the options are `(blank, theirs, mine)` respectively. You want to create 2 new tensors of shape `[d_model, row, col]` where each one is the probe direction for a particular concept (the "blank" concept which we're defining as `blank - (mine + theirs)/2`, and the "mine vs theirs" concept which we're defining as `mine - theirs`). So you want to slice `linear_probe` along its last dimension to create these two tensors.

</details>

In [19]:
# YOUR CODE HERE - define `blank_probe` and `my_probe`, from linear combinations of `linear_probe`
# 0:empty, 1:theirs, 2:mine
blank_probe = linear_probe[:,:,:,0] - (linear_probe[:,:,:,1] + linear_probe[:,:,:,2]) / 2
my_probe = linear_probe[:,:,:,2] - linear_probe[:,:,:,1]

tests.test_my_probes(blank_probe, my_probe, linear_probe)

All tests in `test_my_probes` passed!


<details><summary>Solution</summary>

```python
# blank(0) - (theirs(1) + mine(2))/2
blank_probe = linear_probe[..., 0] - linear_probe[..., 1] * 0.5 - linear_probe[..., 2] * 0.5
# mine(2) - theirs(1)
my_probe = linear_probe[..., 2] - linear_probe[..., 1]
```
</details>

Now that we've got our probe working, let's see it in action!

We'll take the 20th move in the 0th game as our example:

In [20]:
game_index = 0
move = 20

# Plot board state
utils.plot_board_values(
    focus_states[game_index, move],
    title="Focus game states",
    width=400,
    height=400,
    text=focus_legal_moves_annotation[game_index][move],
)

# Plot model predictions
logprobs = t.full(size=(8, 8), fill_value=-13.0, device=device)
logprobs.flatten()[ALL_SQUARES] = focus_logits[game_index, move].log_softmax(dim=-1)[1:]
utils.plot_board_values(logprobs, title=f"Logprobs after move {move}", width=450, height=400)

Now, how does the game state (i.e. which moves are legal for white) change when `F4` is flipped from black to white?

<details>
<summary>Hint</summary>

One move becomes legal, one becomes illegal.

</details>

<details>
<summary>Answer</summary>

- `G4` becomes illegal, because you're no longer surrounding the vertical line of black pieces in column 4.
- `D2` becomes legal, because now you'd be diagonally surrounding the single black piece at `E3`.

</details>

Let's verify this, using the `OthelloBoardState` class:

In [21]:
cell_r = 5
cell_c = 4
print(f"Flipping the color of cell {'ABCDEFGH'[cell_r]}{cell_c}")

board = utils.OthelloBoardState()
board.update(focus_games_square[game_index, : move + 1].tolist())
valid_moves = board.get_valid_moves()
flipped_board = copy.deepcopy(board)
flipped_board.state[cell_r, cell_c] *= -1
flipped_legal_moves = flipped_board.get_valid_moves()

newly_legal = [utils.square_to_label(move) for move in flipped_legal_moves if move not in valid_moves]
newly_illegal = [utils.square_to_label(move) for move in valid_moves if move not in flipped_legal_moves]
print("newly_legal", newly_legal)
print("newly_illegal", newly_illegal)

Flipping the color of cell F4
newly_legal ['D2']
newly_illegal ['G4']


We can now intervene on the model's residual stream using the "my colour vs their colour" direction. I get the best results intervening after layer 4. This is a **linear intervention** - we are just changing a single dimension of the residual stream and keeping the others unchanged. This is a fairly simple intervention, and it's striking that it works!

I apply the fairly janky technique of taking current coordinate in the given direction, negating it, and then multiply by a hyperparameter called `scale` (scale between 1 and 8 tends to work best - small isn't enough and big tends to break things). I haven't tried hard to optimise this and I'm sure it can be improved! Eg by replacing the model's coordinate by a constant rather than scaling it. I also haven't dug into the best scale parameters, or which ones work best in which contexts - plausibly different cells have different activation scales on their world models and need different behaviour!

### Exercise - define the `apply_scale` function

> ```yaml
> Difficulty: 🔴🔴🔴⚪⚪
> Importance: 🔵🔵🔵⚪⚪
>
> You should spend up to 10-20 minutes on this exercise.
> ```

Define a function which will take in the residual stream value and the associated hook point as arguments, and return a modified version of the residual stream, in the way described above.

To be clear, if we define $\vec{v}$ as the probe's flip direction for a given square (called `flip_dir` below), then we can write our residual stream (at `pos=20`, which is the one we're interested in) as the vector:

$$
\text{resid}_{20} = \alpha \times \vec{v} + \beta \times \vec{w}
$$

where $\vec{w}$ is some vector orthogonal to $v$. We want to alter the residual stream at this position to be:

$$
\text{resid}_{20} = - \text{ scale} \times \alpha \times \vec{v} + \beta \times \vec{w}
$$

Remember to normalize vector $\vec{v}$!

In [22]:
def apply_scale(
    resid: Float[Tensor, "batch seq d_model"],
    flip_dir: Float[Tensor, "d_model"],
    scale: int,
    pos: int,
) -> Float[Tensor, "batch seq d_model"]:
    """
    Returns a version of the residual stream, modified by the amount `scale` in the
    direction `flip_dir` at the sequence position `pos`, in the way described above.
    """
    # norm flip_dir
    flip_dir_normed = flip_dir / flip_dir.norm()

    # get dot product
    resid_flip_dir = resid[:,pos,:] @ flip_dir_normed[:,None] # [batch, 1]

    # get orthogonal vector
    resid_ortho_dir = resid[:,pos,:] - (resid_flip_dir * flip_dir_normed[None,:])

    # add -scale * v to orthogonal vector
    resid[:,pos,:] = resid_ortho_dir + ((-scale) * resid_flip_dir * flip_dir_normed[None,:])

    return resid


tests.test_apply_scale(apply_scale)

All tests in `test_apply_scale` passed!


Now, you can run the code below to see the output of your interventions. You should see that the model's prediction changes, it starts predicting `D2` as legal and `G4` as illegal.

In [23]:
flip_dir = my_probe[:, cell_r, cell_c]

logprobs_flipped = []
layer = 4
scales = [0, 1, 2, 4, 8, 16]

# Iterate through scales, generate a new facet plot for each possible scale
for scale in scales:
    # Hook function which will perform flipping in the "F4 flip direction"
    def flip_hook(resid: Float[Tensor, "batch seq d_model"], hook: HookPoint):
        return apply_scale(resid, flip_dir, scale, move)

    # Calculate the logits for the board state, with the `flip_hook` intervention (note that we only
    # need to use :move+1 as input, because of causal attention)
    flipped_logits = model.run_with_hooks(
        focus_games_id[game_index : game_index + 1, : move + 1],
        fwd_hooks=[
            (get_act_name("resid_post", layer), flip_hook),
        ],
    ).log_softmax(dim=-1)[0, move]

    logprobs_flipped_single = t.zeros((64,), dtype=t.float32, device=device) - 10.0
    logprobs_flipped_single[ALL_SQUARES] = flipped_logits.log_softmax(dim=-1)[1:]
    logprobs_flipped.append(logprobs_flipped_single)

flip_state_big = t.stack(logprobs_flipped)
logprobs_repeated = einops.repeat(logprobs.flatten(), "d -> b d", b=6)
color = t.zeros((len(scales), 64)) + 0.2
color[:, utils.to_square(newly_legal)] = 1
color[:, utils.to_square(newly_illegal)] = -1

scatter(
    y=logprobs_repeated,
    x=flip_state_big,
    title=f"Original vs Flipped {utils.square_to_label(8 * cell_r + cell_c)} at Layer {layer}",
    xaxis="Flipped",
    yaxis="Original",
    hover=[f"{r}{c}" for r in "ABCDEFGH" for c in range(8)],
    facet_col=0,
    facet_labels=[f"Translate by {i}x" for i in scales],
    color=color,
    color_name="Newly Legal",
    color_continuous_scale="Geyser",
    width=1400,
)

<details>
<summary>Help - I'm still confused about this figure / this methodology.</summary>

When we mean "translating by `Nx`" for a scalar `N`, we mean "taking the component `x` of the residual stream in the `theirs - mine` probe direction for the F4 square, and replacing it with `Nx`.

The scatter plots compare `-1x` (original) to `Nx` (flipped) for different values of `x`. For instance, the first facet plot shows what happens when the residual stream's component in the probe direction is erased.

The fact that we see the model's predictions for `G4` and `D2` change (with `G4` "becoming more illegal" and `D2` "becoming more legal") as our scale factor increases (without significant change in the predictions for other squares, at first) is evidence that our caual intervention is valid. In other words, the direction found by our linear probe `my_probe` does in some sense represent the model's `theirs - mine` direction, and this direction is used by the model downstream.
</details>

<br>

> #### Final recap of the useful objects we've defined
>
> We have the following:
>
> Models
> - `model` is an 8-layer autoregressive transformer, trained to predict legal Othello moves. Its vocab is `range(0, 61)` where 0 = "pass" and the other numbers represent the 60 possible moves, excluding the 4 middle squares
> - `full_linear_probe.shape = (mode=3, d_model=512, row=8, col=8, options=3)`, probe in the "black/white basis" (modes are black/white/both to play, options are empty/white/black)
> - `linear_probe.shape = (d_model=512, row=8, col=8, options=3)`, probe in the "theirs/mine basis" (options are empty/theirs/mine)
> - `blank_probe` and `my_probe`, both have shape `(d_model=512, row=8, col=8)` and are created from linear combinations of the `linear_probe` options
>
> All data
> - `board_seqs_id.shape = (100k, 60)` contains the moves from all 100k games (as token ids)
> - `board_seqs_square.shape = (100k, 60)` contains the moves from all 100k games (as ints)
>
> Focus games data
> - `focus_games_id.shape = (50, 60)` contains the moves from 50 games (as token ids)
> - `focus_games_square.shape = (50, 60)` contains the moves from 50 games (as ints)
> - `focus_states.shape = (50, 60, 8, 8)` contains the board states after each move (0 = empty, 1 = black, -1 = white)
> - `focus_legal_moves.shape = (50, 60, 8, 8)` contains a 1 for each legal move, and 0 for each illegal move
> - `focus_logits.shape = (50, 59, 61)` contains model's output logits on focus games (59 because we don't include the final move in our fwd pass, 61 because our vocab size is 61 - we have 60 moves + 1 for pass)

# 2️⃣ Looking for modular circuits

> ##### Learning Objectives
>
> - Learn how to use our linear probe across multiple layers
> - Apply activation patching at a given sequence position to test hypotheses about our model
> - Understand how a neuron can be characterized in terms of its input and output weights

## Probing Across Layers

The probe's inputs are accumulated in the residual stream over the six layers - the residual stream is the sum of the output of each previous head and neuron. We can therefore analyse which previous model components most contribute to the overall probe computation, and use this to identify the end of the world model computing circuit.

Let's analyse move 20 in game 1, where we can see that the probe has perfect accuracy after layer 6.

In [24]:
layer = 6
game_index = 1
move = 20

utils.plot_board_values(
    focus_states[game_index, move],
    text=focus_legal_moves_annotation[game_index][move],
    title=f"Focus game #{game_index}, board after move {move}",
    width=400,
    height=400,
)

plot_probe_outputs(focus_cache, linear_probe, layer, game_index, move, title=f"Probe outputs (layer {layer})")

We now plot the contributions of the attention and MLP layers to the `my_probe` direction. Strikingly, we see that the MLP layers are important for the vertical stripe that we just taken by the opponent, but that most of the rest seems to be done by the attention layers.

### Exercise - compute attn and mlp contributions

> ```yaml
> Difficulty: 🔴🔴🔴⚪⚪
> Importance: 🔵🔵🔵🔵⚪
>
> You should spend up to 10-20 minutes on this exercise.
> This is an important exercise to be able to do - logit attribution is one of the most important tools in the interpretability toolbox.
> ```

Below, you should define `attn_contributions` and `mlp_contributions`. You should do this by taking the batched dot product of the vectors *written* to the residual stream in each layer (from layer `0` to layer `layer` inclusive), and the probe direction "my vs their" which you computed earlier (i.e. `my_probe`).

Note, we're looking for the marginal contribution to the probe direction from each of our components, not the accumulated residual stream. This is because we want to see which components have a strong effect on the output.

<details>
<summary>Hint - what activation names to use?</summary>

You should be using `attn_out` and `mlp_out`.

Calculating each of these two contributions will require taking an `einsum` with the activations and your probe.
</details>

<details>
<summary>Hint - what dimensions to multiply over?</summary>

`my_probe` has shape `(d_model=512, rows=8, cols=8)`. You should multiply the residual stream at a given layer and `my_probe` over the `d_model` dimension (since your probe represents directions in the residual stream). Your output (for a given game index and move) will have shape `(rows=8, cols=8)`, and will represent the amount by which that component of your model writes to the residual stream in the probe directions.
</details>

In [42]:
def calculate_attn_and_mlp_probe_score_contributions(
    focus_cache: ActivationCache,
    probe: Float[Tensor, "d_model rows cols"],
    layer: int,
    game_index: int,
    move: int,
) -> tuple[Float[Tensor, "layers rows cols"], Float[Tensor, "layers rows cols"]]:
    # YOUR CODE HERE - define `attn_contributions` and `mlp_contributions` using the cache & probe

    attn_contributions = t.empty(layer+1, 8, 8)
    mlp_contributions = t.empty(layer+1, 8, 8)

    for l in range(layer+1):
        attn_out = focus_cache["attn_out",l][game_index, move, :] #[d_model]
        mlp_out = focus_cache["mlp_out",l][game_index, move, :] #[d_model]

        attn_contributions[l,:,:] = einops.einsum(attn_out, probe, "d_model, d_model row col -> row col")
        mlp_contributions[l,:,:] = einops.einsum(mlp_out, probe, "d_model, d_model row col -> row col")

    return (attn_contributions, mlp_contributions)


layer = 6
attn_contributions, mlp_contributions = calculate_attn_and_mlp_probe_score_contributions(
    focus_cache, my_probe, layer, game_index, move
)

utils.plot_board_values(
    mlp_contributions,
    title=f"MLP Contributions to my vs their (game #{game_index}, move {move})",
    board_titles=[f"Layer {i}" for i in range(layer + 1)],
    width=1400,
    height=340,
)
utils.plot_board_values(
    attn_contributions,
    title=f"Attn Contributions to my vs their (game #{game_index}, move {move})",
    board_titles=[f"Layer {i}" for i in range(layer + 1)],
    width=1400,
    height=340,
)

<details><summary>Solution</summary>

```python
def calculate_attn_and_mlp_probe_score_contributions(
    focus_cache: ActivationCache,
    probe: Float[Tensor, "d_model rows cols"],
    layer: int,
    game_index: int,
    move: int,
) -> tuple[Float[Tensor, "layers rows cols"], Float[Tensor, "layers rows cols"]]:
    attn_contributions = einops.einsum(
        t.stack([focus_cache["attn_out", l][game_index, move] for l in range(layer + 1)]),
        probe,
        "layers d_model, d_model rows cols -> layers rows cols",
    )
    mlp_contributions = einops.einsum(
        t.stack([focus_cache["mlp_out", l][game_index, move] for l in range(layer + 1)]),
        probe,
        "layers d_model, d_model rows cols -> layers rows cols",
    )

    return (attn_contributions, mlp_contributions)
```
</details>

Next, you should return (and plot) overall probe scores (i.e. from the accumulated residual stream by the end of layer `layer`). The code should be similar to the code you wrote immediately above, except you're taking the value at the residual stream up to and including layer `layer`, rather than the MLP or attention contributions.

In [39]:
def calculate_accumulated_probe_score(
    focus_cache: ActivationCache,
    probe: Float[Tensor, "d_model rows cols"],
    layer: int,
    game_index: int,
    move: int,
) -> Float[Tensor, "layers rows cols"]:

    resid_streams = t.stack([focus_cache["resid_post", l][game_index, move, :] for l in range(layer+1)], dim=0) #[layer d_model]
    residual_stream_score = einops.einsum(resid_streams, probe, "layer d_model, d_model row col -> layer row col")

    return residual_stream_score


residual_stream_score = calculate_accumulated_probe_score(focus_cache, my_probe, layer, game_index, move)

utils.plot_board_values(
    residual_stream_score,
    title=f"Residual stream probe values for 'my vs their' (game #{game_index}, move {move})",
    board_titles=[f"Layer {i}" for i in range(layer + 1)],
    width=1400,
    height=340,
)

<details><summary>Solution</summary>

```python
def calculate_accumulated_probe_score(
    focus_cache: ActivationCache,
    probe: Float[Tensor, "d_model rows cols"],
    layer: int,
    game_index: int,
    move: int,
) -> Float[Tensor, "layers rows cols"]:
    residual_stream_score = einops.einsum(
        t.stack([focus_cache["resid_post", l][game_index, move] for l in range(layer + 1)]),
        probe,
        "layer d_model, d_model rows cols -> layer rows cols",
    )

    return residual_stream_score
```
</details>

### Exercise - repeat this for the "blank" probe

> ```yaml
> Difficulty: 🔴⚪⚪⚪⚪
> Importance: 🔵🔵🔵⚪⚪
>
> You should spend ~5-10 minutes on this exercise - it's just repeating previous code.
> ```

Make exactly the same plots, but using `blank_probe` instead of `my_probe`. What do you notice, and why?

In [40]:
# YOUR CODE HERE - repeat the results for `blank_probe`, and interpret them
layer = 6
attn_contributions, mlp_contributions = calculate_attn_and_mlp_probe_score_contributions(
    focus_cache, blank_probe, layer, game_index, move
)

# attn, mlp contributions
utils.plot_board_values(
    mlp_contributions,
    title=f"MLP Contributions to my vs their (game #{game_index}, move {move})",
    board_titles=[f"Layer {i}" for i in range(layer + 1)],
    width=1400,
    height=340,
)
utils.plot_board_values(
    attn_contributions,
    title=f"Attn Contributions to my vs their (game #{game_index}, move {move})",
    board_titles=[f"Layer {i}" for i in range(layer + 1)],
    width=1400,
    height=340,
)


residual_stream_score = calculate_accumulated_probe_score(focus_cache, blank_probe, layer, game_index, move)

# total layer contributions
utils.plot_board_values(
    residual_stream_score,
    title=f"Residual stream probe values for 'my vs their' (game #{game_index}, move {move})",
    board_titles=[f"Layer {i}" for i in range(layer + 1)],
    width=1400,
    height=340,
)

<details>
<summary>Solution code</summary>

```python
attn_contributions, mlp_contributions = calculate_attn_and_mlp_probe_score_contributions(
    focus_cache, blank_probe, layer, game_index, move
)
utils.plot_board_values(
    mlp_contributions,
    title=f"MLP Contributions to blank probe (game #{game_index}, move {move})",
    board_titles=[f"Layer {i}" for i in range(layer + 1)],
    width=1400,
    height=340,
)
utils.plot_board_values(
    attn_contributions,
    title=f"Attn Contributions to blank probe (game #{game_index}, move {move})",
    board_titles=[f"Layer {i}" for i in range(layer + 1)],
    width=1400,
    height=340,
)

residual_stream_score = calculate_accumulated_probe_score(focus_cache, blank_probe, layer, game_index, move)
utils.plot_board_values(
    residual_stream_score,
    title=f"Residual stream probe values for 'blank' (game #{game_index}, move {move})",
    board_titles=[f"Layer {i}" for i in range(layer + 1)],
    width=1400,
    height=340,
)
```

</details>

<details>
<summary>Discussion</summary>

The algorithm "is this cell blank or not?" is pretty easy to implement - you just need to check if that cell has been played at some point in the game. Unlike telling whether a cell is black or white, this doesn't require an understanding of the piece-flipping rules.

The model seems to have a pretty good understanding of this past the zeroth attention layer, or at least it has all the pieces to understand that (even the larger magnitude contribution comes from e.g. the later MLP layers).

</details>

## Reading off neuron weights

Another cool consequence of having a linear probe is having an interpretable set of directions in the residual stream. This means that we can read off the meaning of any neuron's input and output weights, in terms of the set of directions given by the probe.

Let's start with neuron L5N1393, which seemed interesting from my initial investigations.

Firstly, we'll compute the normalized version of the probes (normalization happens over the `d_model` dimension, i.e. `blank_probe_normalized` has shape `(d_model, row, col)` and the `[:, i, j]`-th entry is the residual stream probe direction fort the `i, j`-th cell in the board).

In [43]:
# Scale the probes down to be unit norm per cell
blank_probe_normalised = blank_probe / blank_probe.norm(dim=0, keepdim=True)
my_probe_normalised = my_probe / my_probe.norm(dim=0, keepdim=True)

# Set the center blank probes to 0, since they're never blank so the probe is meaningless
blank_probe_normalised[:, [3, 3, 4, 4], [3, 4, 3, 4]] = 0.0

### Exercise - calculate neuron input weights

> ```yaml
> Difficulty: 🔴🔴🔴⚪⚪
> Importance: 🔵🔵🔵🔵⚪
>
> You shouldn't spend more than 10-25 minutes on this exercise.
> ```

The function `calculate_neuron_input_weights` below takes `layer` and `neuron`. It should return a tensor of shape `(row, col)`, where the `[i, j]`-th entry is the projection of this neuron's input weights onto the probe direction corresponding to the `i, j`-th cell in the board.

The function `calculate_neuron_output_weights` is very similar, but returns the projection of the neuron's output weights onto the probe direction.

Recall that, when we talk about a neuron's input and output weights, we're referring to the following decomposition:

$$
\begin{align*}
f(x) &= f(x^T W^{in}) W^{out} \\
&= \sum_n f(x^T W^{in}_{[:, n]}) W^{out}_{[n, :]}
\end{align*}
$$

where $x$ is a vector in the residual stream, $W^{in}$ is the input weight matrix, $W^{out}$ is the output weight matrix, $f$ is the activation function, and $\sum_n$ represents a sum over neurons.

You'll first write the helper function `get_w_in` and `get_w_out`, which returns the (normalized) vectors $W^{in}_{[:, n]}$ and $W^{out}_{[n, :]}$ for a given neuron. Then, you'll implement `calculate_neuron_input_weights` using this helper function.

Why do we normalize before projecting onto the probe direction? The reason we do this is because we don't care about the scale factor - you could double the magnitude of the output vector and half that of the corresponding input vector, and (ignoring biases) the result would be the same. Instead, we care about how much the input direction of our model's weights aligns with the probe direction we found in the residual stream. The fact that we've also normalized our probes means that we'll be plotting the **cosine similarity** of vectors.

*Note - remember to use `clone()` and `detach()` if you're indexing into a model's weights and performing operations on it, when necessary. You use `clone()` because you don't want to modify the model's weights, and `detach()` because you don't want to compute gradients through the model's weights.*

In [58]:
def get_w_in(
    model: HookedTransformer,
    layer: int,
    neuron: int,
    normalize: bool = False,
) -> Float[Tensor, "d_model"]:
    """
    Returns the input weights for the given neuron.

    If normalize is True, the weight is normalized to unit norm.
    """
    ret_vec = model.W_in[layer,:,neuron].detach().clone()
    return ret_vec / ret_vec.norm() if normalize else ret_vec


def get_w_out(
    model: HookedTransformer,
    layer: int,
    neuron: int,
    normalize: bool = False,
) -> Float[Tensor, "d_model"]:
    """
    Returns the output weights for the given neuron.

    If normalize is True, the weight is normalized to unit norm.
    """
    ret_vec = model.W_out[layer,neuron,:].detach().clone()
    return ret_vec / ret_vec.norm() if normalize else ret_vec


def calculate_neuron_input_weights(
    model: HookedTransformer, probe: Float[Tensor, "d_model row col"], layer: int, neuron: int
) -> Float[Tensor, "rows cols"]:
    """
    Returns tensor of the input weights for the given neuron, at each square on the board, projected
    along the corresponding probe directions.

    Assume probe directions are normalized. You should also normalize the model weights.
    """
    return einops.einsum(get_w_in(model, layer, neuron, normalize=True), probe, "d_model, d_model row col -> row col")


def calculate_neuron_output_weights(
    model: HookedTransformer, probe: Float[Tensor, "d_model row col"], layer: int, neuron: int
) -> Float[Tensor, "rows cols"]:
    """
    Returns tensor of the output weights for the given neuron, at each square on the board,
    projected along the corresponding probe directions.

    Assume probe directions are normalized. You should also normalize the model weights.
    """
    return einops.einsum(get_w_out(model, layer, neuron, normalize=True), probe, "d_model, d_model row col -> row col")


tests.test_calculate_neuron_input_weights(calculate_neuron_input_weights, model)
tests.test_calculate_neuron_output_weights(calculate_neuron_output_weights, model)

All tests in `test_calculate_neuron_input_weights` passed!
All tests in `test_calculate_neuron_output_weights` passed!


<details><summary>Solution</summary>

```python
def get_w_in(
    model: HookedTransformer,
    layer: int,
    neuron: int,
    normalize: bool = False,
) -> Float[Tensor, "d_model"]:
    """
    Returns the input weights for the given neuron.

    If normalize is True, the weight is normalized to unit norm.
    """
    w_in = model.W_in[layer, :, neuron].detach().clone()
    if normalize:
        w_in /= w_in.norm(dim=0, keepdim=True)
    return w_in


def get_w_out(
    model: HookedTransformer,
    layer: int,
    neuron: int,
    normalize: bool = False,
) -> Float[Tensor, "d_model"]:
    """
    Returns the output weights for the given neuron.

    If normalize is True, the weight is normalized to unit norm.
    """
    w_out = model.W_out[layer, neuron, :].detach().clone()
    if normalize:
        w_out /= w_out.norm(dim=0, keepdim=True)
    return w_out


def calculate_neuron_input_weights(
    model: HookedTransformer, probe: Float[Tensor, "d_model row col"], layer: int, neuron: int
) -> Float[Tensor, "rows cols"]:
    """
    Returns tensor of the input weights for the given neuron, at each square on the board, projected
    along the corresponding probe directions.

    Assume probe directions are normalized. You should also normalize the model weights.
    """
    w_in = get_w_in(model, layer, neuron, normalize=True)

    return einops.einsum(w_in, probe, "d_model, d_model row col -> row col")


def calculate_neuron_output_weights(
    model: HookedTransformer, probe: Float[Tensor, "d_model row col"], layer: int, neuron: int
) -> Float[Tensor, "rows cols"]:
    """
    Returns tensor of the output weights for the given neuron, at each square on the board,
    projected along the corresponding probe directions.

    Assume probe directions are normalized. You should also normalize the model weights.
    """
    w_out = get_w_out(model, layer, neuron, normalize=True)

    return einops.einsum(w_out, probe, "d_model, d_model row col -> row col")
```
</details>

Now, let's examine neuron `1393` in more detail. Can you interpret what it's doing?

In [59]:
layer = 5
neuron = 1393

w_in_L5N1393_blank = calculate_neuron_input_weights(model, blank_probe_normalised, layer, neuron)
w_in_L5N1393_my = calculate_neuron_input_weights(model, my_probe_normalised, layer, neuron)

utils.plot_board_values(
    t.stack([w_in_L5N1393_blank, w_in_L5N1393_my]),
    title=f"Input weights in terms of the probe for neuron L{layer}N{neuron}",
    board_titles=["Blank In", "My In"],
    width=650,
    height=380,
)

<details>
<summary>Answer - what this neuron is doing</summary>

It seems to represent `(C0==BLANK) & (D1==THEIRS) & (E2==MINE)` - in other words, it fires strongest when all three of these conditions hold.

This is useful for the model, because if all three of these conditions hold, then `C0` is a legal move (because it flips `D1`).

</details>

### Exercise - test your hypothesis (output behaviour)

> ```yaml
> Difficulty: 🔴🔴🔴⚪⚪
> Importance: 🔵🔵🔵⚪⚪
>
> You should spend up to 10-20 minutes on this exercise.
> ```

The plots above are evidence about the neuron's input behaviour (i.e. what causes it to fire), but we haven't tested its output behaviour (i.e. the effect it has when it fires) to see if it's in line with our hypothesis. You should make a particular plot to test your hypothesis about the neuron, using the `plot_board_values` function - we'll also leave it as an exercise to figure out what plot you should make!

In [77]:
# YOUR CODE HERE - create a plot to test your prediction
w_out_L5N1393 = get_w_out(model, layer, neuron, normalize=True)
W_U_normed = model.W_U[:,1:] / model.W_U[:,1:].norm(dim=0,keepdim=True)

cos_sim = w_out_L5N1393 @ W_U_normed

L5N1393_board = t.zeros(size=(8, 8), device=device)
L5N1393_board.flatten()[ALL_SQUARES] = w_out_L5N1393_unembed[1:]  # the [1:] is to filter out logits for the "pass" move

utils.plot_board_values(
    L5N1393_board[None,:,:],
    board_titles=["unembed neuron output weight"],
    width=450,
    height=380,
)

<details>
<summary>Answer - what plot you should make</summary>

You should expect that this neuron fires to predict `C0` is legal. In other words, when you map its output weights through the unembedding matrix, the values should be large for `C0` and small for the other cells. You can test this by taking the output weights, mapping them through the unembedding, then plotting the result in an 8x8 square.

The name for this is **direct logit attribution**, or DLA. This is the standard way we study the direct effect of components that write to the residual stream (i.e. ignoring the paths that go through intermediate components).

</details>

<details>
<summary>Solution (code for plot)</summary>

Run the code below to create the plot - you should find that the neuron does boost the logit for `C0` when it fires. Interestingly, there's also a positive logit effect on the cell `D1`. We'll look at this a bit deeper later on in the exercises, when we take a deeper dive into DLA.

```python
# Get neuron output weights' cos sim with unembedding
w_out_L5N1393 = get_w_out(model, layer, neuron, normalize=True)
W_U_normalized = model.W_U[:, 1:] / model.W_U[:, 1:].norm(dim=0, keepdim=True)  # normalize, slice off logits for "pass"
cos_sim = w_out_L5N1393 @ W_U_normalized  # shape (60,)

# Turn into a (rows, cols) tensor, using indexing
cos_sim_rearranged = t.zeros((8, 8), device=device)
cos_sim_rearranged.flatten()[ALL_SQUARES] = cos_sim

# Plot results
utils.plot_board_values(
    cos_sim_rearranged,
    title=f"Cosine sim of neuron L{layer}N{neuron} with W<sub>U</sub> directions",
    width=450,
    height=380,
)
```

Note - you can also choose not to normalize weights; this will show you the output weights in the unembedding basis, rather than the cosine similarity between neuron output weight & unembedding vectors. But the latter is more informative because we know that 1 is the maximum value (and we have an idea of what to expect - the expected absolute value of the cosine similarity of 2 randomly chosen vectors in ND space scales as `1/sqrt(N)` (we omit the derivation), which means `1/sqrt(512) ≈ 0.04` - you can check this empirically if you'd like).

</details>

### How much variance does the probe explain?

We can also look at what fraction of the neuron's input and output weights are captured by the probe (because the vector was scaled to have unit norm, looking at the squared norm of its projection gives us this answer).

We see that the input weights are well explained by this, while the output weights are only somewhat well explained by this.

In [78]:
w_in_L5N1393 = get_w_in(model, layer, neuron, normalize=True)
w_out_L5N1393 = get_w_out(model, layer, neuron, normalize=True)

U, S, Vh = t.svd(t.cat([my_probe.reshape(cfg.d_model, 64), blank_probe.reshape(cfg.d_model, 64)], dim=1))

# Remove the final four dimensions of U, as the 4 center cells are never blank and so the blank
# probe is meaningless there.
probe_space_basis = U[:, :-4]

print(f"Fraction of input weights in probe basis: {((w_in_L5N1393 @ probe_space_basis).pow(2).sum()):.4f}")
print(f"Fraction of output weights in probe basis: {((w_out_L5N1393 @ probe_space_basis).pow(2).sum()):.4f}")

Fraction of input weights in probe basis: 0.6818
Fraction of output weights in probe basis: 0.1633


<details>
<summary>Help - I don't understand what's going on here.</summary>

The concatenated probe directions collectively have rank 128, just 1/4 of the total model dimensionality `d_model=512`. For a randomly chosen vector, we might expect about 1/4 of its squared norm to lie in the span of these probe directions (or to phrase it a different way, we might expect the probe directions to explain about 1/4 of the vector's variance).

For the input weights, we see that this value is larger, i.e. the neuron seems to fire primarily in response to these probe directions (this agrees with what we saw above; that the neuron was detecting particular board states associated with these probe directions). For the output weights, this value is smaller, which would also make sense if our neuron was mainly predicting "C0 is legal" rather than itself being used to update the board state. In later exercises, you'll show that this is the case!

</details>

### More neurons

Lets try this on the layer 3 neurons with the top standard deviation (of activations), and look at how their output weights affect the my probe direction.

In [79]:
layer = 3
top_neurons = focus_cache["post", layer][:, 3:-3].std(dim=[0, 1]).argsort(descending=True)[:10]

utils.plot_board_values(
    t.stack([calculate_neuron_output_weights(model, blank_probe_normalised, layer, n) for n in top_neurons]),
    title=f"Cosine sim of output weights and the 'blank color' probe for top layer {layer} neurons (by std dev)",
    board_titles=[f"L{layer}N{n.item()}" for n in top_neurons],
    width=1600,
    height=360,
)

utils.plot_board_values(
    t.stack([calculate_neuron_output_weights(model, my_probe_normalised, layer, n) for n in top_neurons]),
    title=f"Cosine sim of output weights and the 'my color' probe for top layer {layer} neurons (by std dev)",
    board_titles=[f"L{layer}N{n.item()}" for n in top_neurons],
    width=1600,
    height=360,
)

Note - you can also experiment with **kurtosis** instead, which is a measure of the tail extremity of our distribution. This might help you find more neurons which activate sparsely (with signficiant outlier values) rather than just identifying neurons with a good mix of both small and large values (for more on why sparsity might tend to correlate with interpretability, see the material in this chapter on sparse autoencoders!).

<details>
<summary>Use this dropdown to get some code to implement kurtosis</summary>

```python
def kurtosis(tensor: Tensor, reduced_axes, fisher=True):
    """
    Computes the kurtosis of a tensor over specified dimensions.
    """
    return (
        ((tensor - tensor.mean(dim=reduced_axes, keepdim=True)) / tensor.std(dim=reduced_axes, keepdim=True)) ** 4
    ).mean(dim=reduced_axes, keepdim=False) - fisher * 3


top_layer_3_neurons = einops.reduce(
    focus_cache["post", layer][:, 3:-3], "game move neuron -> neuron", reduction=kurtosis
).argsort(descending=True)[:10]
```

</details>

We can also try plotting the top neurons for layer 4:

In [80]:
layer = 4
top_neurons = focus_cache["post", layer][:, 3:-3].std(dim=[0, 1]).argsort(descending=True)[:10]

utils.plot_board_values(
    t.stack([calculate_neuron_output_weights(model, blank_probe_normalised, layer, n) for n in top_neurons]),
    title=f"Cosine sim of output weights and the 'blank color' probe for top layer {layer} neurons (by std dev)",
    board_titles=[f"L{layer}N{n.item()}" for n in top_neurons],
    width=1600,
    height=360,
)

utils.plot_board_values(
    t.stack([calculate_neuron_output_weights(model, my_probe_normalised, layer, n) for n in top_neurons]),
    title=f"Cosine sim of output weights and the 'my color' probe for top layer {layer} neurons (by std dev)",
    board_titles=[f"L{layer}N{n.item()}" for n in top_neurons],
    width=1600,
    height=360,
)

Why do all the top layer 4 neurons have such striking results, with almost perfect alignment with one of the blank probe directions (and very low alignment with the "my color" probe directions)?

A cell can *only* be legal to play in if it is blank (obviously). Since calculating blankness is easy (you just check whether a move was played), the model should be able to do this in a single layer - these are the neurons that we see above.

**Question - if this is true, then what observation should we expect when we compare the neuron output weights to the unembedding weights?**

Think about this before you read on.

<details>
<summary>Answer</summary>

If this is true, then we should expect the cosine similarity of output weights and the unembedding weights to exhibit the same heatmap pattern as we see above.

In other words, these neurons which are firing strongly on blank cells are also directly writing to the residual stream, having their output used in the unembedding to increase the logit score for the blank cells which they are detecting.

</details>

We'll test this out, using the same method of plotting the direct logit attribution as you should have used for the exercise "test your hypothesis" above.

In [81]:
layer = 4
top_neurons = focus_cache["post", layer][:, 3:-3].std(dim=[0, 1]).argsort(descending=True)[:10]
w_out = t.stack([get_w_out(model, layer, neuron, normalize=True) for neuron in top_neurons])

# Get neuron output weights' cos sim with unembedding
W_U_normalized = model.W_U[:, 1:] / model.W_U[:, 1:].norm(dim=0, keepdim=True)  # normalize, slice off logits for "pass"
cos_sim = w_out @ W_U_normalized

# Turn into a tensor, using indexing
cos_sim_rearranged = t.zeros((10, 8, 8), device=device)
cos_sim_rearranged.flatten(1, -1)[:, ALL_SQUARES] = cos_sim

# Plot results
utils.plot_board_values(
    cos_sim_rearranged,
    title=f"Cosine sim of top neurons with W<sub>U</sub> directions (layer {layer})",
    board_titles=[f"L{layer}N{n.item()}" for n in top_neurons],
    width=1500,
    height=320,
)

Great! We've validated our hypothesis that the neurons in this layer are directly computing "blankness" and feeding this output directly into the unembedding, in order to raise the logit score for blank cells (since being blank is necessary for being legal to play in).

In section 3️⃣, we'll do more of this kind of direct logit attribution.


<details>
<summary>Question - if you try plotting the top layer 4 neurons by kurtosis, you won't find these "blank detecting" neurons (even though kurtosis seems to give better results for layer 3). Why do you think this is?</summary>

Sorting by kurtosis is best at finding outliers. Sorting by std dev will find you neurons with good variability / spread, i.e. a good mix of small and large values. This latter description better describes the blank detecting neurons, because they'll be bimodal (off if the square is blank, on if it's occupied - and both of these groups are nontrivially sized).


</details>

### Recap of this section

We did the following:

* Defined helper functions `get_w_in`, `get_w_out` which returned particular MLP weight vectors for a given neuron.
* Defined functions `calculate_neuron_input_weights` and `calculate_neuron_output_weights` which returned the projection of a neuron's input and output weights onto the directions of a given probe (e.g. `my_probe` or `blank_probe`).
* Discovered that some neurons have very interpretable input weights, for example:
    * After comparing the input weights of `L5N1393` to the probe directions, we found that it was detecting a particular diagonal line of blank-theirs-mine, since this indicates that the blank square is legal.
        * This is interesting, and not necessarily something we could have predicted beforehand.
        * We also looked at the logit lens for this neuron, and found that it was boosting the logit for the square that would be legal if this pattern was present.
    * After looking at the input weights of some layer 4 neurons, we found that they were detecting whether a square was blank (since this is necessary for legality), and their output weights seemed to be directly used in the embedding to increase log probs for those blank squares.
        * This is not as interesting and probably something we could have predicted, because the "check if cell" is blank operation is pretty easy.

## Activation Patching

A valuable technique for tracking down the action of various circuits is activation patching. This was first introduced in [David Bau and Kevin Meng's excellent ROME paper](https://rome.baulab.info/), there called causal tracing.

The setup of activation patching is to take two runs of the model on two different inputs, the clean run and the corrupted run. The clean run outputs the correct answer and the corrupted run does not. The key idea is that we give the model the corrupted input, but then **intervene** on a specific activation and **patch** in the corresponding activation from the clean run (ie replace the corrupted activation with the clean activation), and then continue the run. And we then measure how much the output has updated towards the correct answer.

By carefully choosing clean and corrupted inputs that differ in one key detail, we can isolate which model components capture and depend on this detail.

The ability to localise is a key move in mechanistic interpretability - if the computation is diffuse and spread across the entire model, it is likely much harder to form a clean mechanistic story for what's going on. But if we can identify precisely which parts of the model matter, we can then zoom in and determine what they represent and how they connect up with each other, and ultimately reverse engineer the underlying circuit that they represent.

The diagrams below demonstrate activation patching on an abstract neural network (the nodes represent activations, and the arrows between them are weight connections).

A regular forward pass on the clean input looks like:

<img src="https://raw.githubusercontent.com/info-arena/ARENA_img/main/misc/simpler-patching-1c.png" width="300">

And activation patching from a corrupted input (green) into a forward pass for the clean input (black) looks like:

<img src="https://raw.githubusercontent.com/info-arena/ARENA_img/main/misc/simpler-patching-2c.png" width="440">

where the dotted line represents patching in a value (i.e. during the forward pass on the clean input, we replace node $D$ with the value it takes on the corrupted input). Nodes $H$, $G$ and $F$ are colored orange, to represent that they now follow a distribution which is not the same as clean or corrupted.

We can patch into a transformer in many different ways (e.g. values of the residual stream, the MLP, or attention heads' output - see below). We can also get even more granular by patching at particular sequence positions (not shown in diagram).

<img src="https://raw.githubusercontent.com/info-arena/ARENA_img/main/misc/simpler-patching-examples.png" width="840">

### Setting up our patching

Before we patch, we need to decide what our clean and corrupted datasets will be, and create a metric for evaluating a set of logits.

Setting up clean and corrupted moves that result in similar games is non-trivial, so here we take a game and just change the most recent move from `E0` to `C0`. This turns `F0` (as a move for white to play) from legal to illegal, so let's make that logit our patching metric.

It also makes sense to have the metric be a linear function of the logit difference. This is enough to uniquely specify a metric.

First, we can plot the original and corrupted boards, to visualize this:

In [84]:
cell_r = 5
cell_c = 4
print(f"Flipping the color of cell {'ABCDEFGH'[cell_r]}{cell_c}")

board = utils.OthelloBoardState()
board.update(focus_games_square[game_index, : move + 1].tolist())
valid_moves = board.get_valid_moves()
flipped_board = copy.deepcopy(board)
flipped_board.state[cell_r, cell_c] *= -1
flipped_legal_moves = flipped_board.get_valid_moves()

newly_legal = [utils.square_to_label(move) for move in flipped_legal_moves if move not in valid_moves]
newly_illegal = [utils.square_to_label(move) for move in valid_moves if move not in flipped_legal_moves]
print("newly_legal", newly_legal)
print("newly_illegal", newly_illegal)

Flipping the color of cell F4
newly_legal ['F7']
newly_illegal ['G4', 'G5']


In [96]:
game_index = 4
move = 20

# Get original & corrupted games (as token IDs & ints)
original_game_id = focus_games_id[game_index, : move + 1]
corrupted_game_id = original_game_id.clone()
corrupted_game_id[-1] = utils.label_to_id("C0")
original_game_square = t.tensor([utils.id_to_square(original_game_id)])
corrupted_game_square = t.tensor([utils.id_to_square(corrupted_game_id)])

original_state, original_legal_moves, original_legal_moves_annotation = get_board_states_and_legal_moves(
    original_game_square
)
corrupted_state, corrupted_legal_moves, corrupted_legal_moves_annotation = get_board_states_and_legal_moves(
    corrupted_game_square
)
utils.plot_board_values(
    t.stack([original_state[0,move], corrupted_state[0,move]], dim=0),
    text=[original_legal_moves_annotation[0][move], corrupted_legal_moves_annotation[0][move]],
    title="Focus game states",
    board_titles=["Original game (black plays E0)", "Corrupted game (black plays C0)"],
    width=650,
    height=380,
)

Next, let's get our logits & cache for both games:

In [97]:
original_logits, original_cache = model.run_with_cache(original_game_id)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_game_id)

original_log_probs = original_logits.log_softmax(dim=-1)
corrupted_log_probs = corrupted_logits.log_softmax(dim=-1)

### Exercise - create a patching metric

> ```yaml
> Difficulty: 🔴🔴⚪⚪⚪
> Importance: 🔵🔵🔵🔵⚪
>
> You shouldn't spend more than 10-15 minutes on this exercise.
> ```

Finally, we'll create a patching metric. This is a function we'll apply to our output logits, in order to measure how much they've changed (in some important way) from their clean values.

We want our patching metric to satisfy the following conditions:

* Should have value **one** when the logits are the same as in the clean distribution.
* Should have value **zero** when the logits are the same as in the corrupted distribution.
    * *Note - we sometimes use the opposite convention. Either might be justified, depending on the context. Here, you should think of a value of 1 meaning "100% of performance is preserved", and 0 as "performance is gone".*
* Should be a **linear** function of the log probs.
    * **Important note** - this is *not* the same as being a linear function of the logits. Can you see why?
* Should just be a function of the logits for the `F0` token, at the final game move (since this is the only move that changes between clean and corrupted).
    * Note - you can index into the `d_vocab` dimension of logits using the `f0_index` variable defined below.

This should be enough for you to uniquely define your patching metric. Also, note that it should return a **scalar tensor** (this is important for the transformerlens patching functions to work).

In [102]:
F0_index = utils.label_to_id("F0")
original_F0_log_prob = original_log_probs[0, -1, F0_index]
corrupted_F0_log_prob = corrupted_log_probs[0, -1, F0_index]

print("Check that the model predicts F0 is legal in original game & illegal in corrupted game:")
print(f"Clean log prob: {original_F0_log_prob.item():.2f}")
print(f"Corrupted log prob: {corrupted_F0_log_prob.item():.2f}\n")


def patching_metric(patched_logits: Float[Tensor, "batch seq d_vocab"]) -> Float[Tensor, ""]:
    """
    Function of patched logits, calibrated so that it equals 0 when performance is same as on
    corrupted input, and 1 when performance is same as on original input.

    Should be linear function of the logits for the F0 token at the final move.
    """
    patched_log_probs = patched_logits.log_softmax(dim=-1)
    return (patched_log_probs[0,-1,F0_index] - corrupted_F0_log_prob) / (original_F0_log_prob - corrupted_F0_log_prob)


tests.test_patching_metric(patching_metric, original_log_probs, corrupted_log_probs)

Check that the model predicts F0 is legal in original game & illegal in corrupted game:
Clean log prob: -2.52
Corrupted log prob: -11.98

All tests in `test_patching_metric` passed!


<details><summary>Solution</summary>

```python
def patching_metric(patched_logits: Float[Tensor, "batch seq d_vocab"]) -> Float[Tensor, ""]:
    """
    Function of patched logits, calibrated so that it equals 0 when performance is same as on
    corrupted input, and 1 when performance is same as on original input.

    Should be linear function of the logits for the F0 token at the final move.
    """
    patched_log_probs = patched_logits.log_softmax(dim=-1)
    return (patched_log_probs[0, -1, F0_index] - corrupted_F0_log_prob) / (original_F0_log_prob - corrupted_F0_log_prob)
```
</details>

### Exercise - write a patching function

> ```yaml
> Difficulty: 🔴🔴🔴🔴⚪
> Importance: 🔵🔵🔵🔵⚪
>
> This exercise is very important; getting it right shows you understand activation patching.
> ```

Below, you should fill in the functions `patch_attn_layer_output` and `patch_mlp_layer_output`.

To do this, you'll have to use TransformerLens hooks. A quick refresher on how they work:

- Hook functions take two compulsory arguments:
    - `tensor` (a PyTorch tensor of the model's activations at this hookpoint)
    - `hook` (a `HookPoint` object, which has helper property `hook.name` and method `hook.layer()`)
- The function `model.run_with_hooks` takes arguments:
    - The tokens to run (as first argument)
    - `fwd_hooks` - a list of `(hook_name, hook_fn)` tuples. Remember that you can use `get_act_name` to get hook names.

Tips:
- It's good practice to have `model.reset_hooks()` at the start of functions which add and run hooks. This is because sometimes hooks fail to be removed (if they cause an error while running). There's nothing more frustrating than fixing a hook error only to get the same error message, not realising that you've failed to clear the broken hook!
- The `HookPoint` object has method `.layer()` and attribute `.name` which can be useful in your hook functions.

In [112]:
def patch_final_move_output(
    activation: Float[Tensor, "batch seq d_model"],
    hook: HookPoint,
    clean_cache: ActivationCache,
) -> Float[Tensor, "batch seq d_model"]:
    """
    Hook function which patches activations at the final sequence position.

    Note, we only need to patch in the final sequence position, because the prior moves in the clean
    and corrupted input are identical (and this is an autoregressive model).
    """
    activation[:,-1,:] = clean_cache[hook.name][:,-1,:]
    return activation


def get_act_patch_resid_pre(
    model: HookedTransformer,
    corrupted_input: Float[Tensor, "batch pos"],
    clean_cache: ActivationCache,
    patching_metric: Callable[[Float[Tensor, "batch seq d_model"]], Float[Tensor, ""]],
) -> Float[Tensor, "2 n_layers"]:
    """
    Returns an array of results corresponding to the results of patching at each (attn_out, mlp_out)
    for all layers in the model.
    """

    metric_vals = t.zeros((2,model.cfg.n_layers))
    partial_hook = partial(patch_final_move_output, clean_cache=clean_cache)

    for layer in range(model.cfg.n_layers):
        model.reset_hooks()

        attn_out_name = f"blocks.{layer}.hook_attn_out"
        patched_attn_output = model.run_with_hooks(corrupted_input, fwd_hooks=[(attn_out_name, partial_hook)])
        patched_attn_metric = patching_metric(patched_attn_output)

        mlp_out_name = f"blocks.{layer}.hook_mlp_out"
        patched_mlp_output = model.run_with_hooks(corrupted_input, fwd_hooks=[(mlp_out_name, partial_hook)])
        patched_mlp_metric = patching_metric(patched_mlp_output)

        metric_vals[:,layer] = t.tensor([patched_attn_metric,patched_mlp_metric])

    return metric_vals


patching_results = get_act_patch_resid_pre(model, corrupted_game_id, original_cache, patching_metric)

pd.options.plotting.backend = "plotly"
pd.DataFrame(to_numpy(patching_results.T), columns=["attn", "mlp"]).plot.line(
    title="Layer Output Patching Effect on F0 Log Prob",
    width=700,
    labels={"value": "Patching Effect", "index": "Layer"},
).show()

<details>
<summary>Spoiler - what results you should get</summary>

We can see that most layers just don't matter! But MLP0, MLP5, MLP6 and Attn7 do! My next steps would be to get more fine grained and to patch in individual neurons and see how far I can zoom in on *why* those layers matter - ideally at the level of understanding how these neurons compose with each other and the changed the embedding, using the fact that most of the model just doesn't matter here. And then to compare this data to the above techniques for understanding neurons. If you want to go off and explore this, that would be a great exercise at this point (or to return to at the end of the exercises).

It's not surprising that the attention layers are fairly unimportant - attention specialises in moving information between token positions, we've only changed the information at the current position! (Attention does have the ability to operate on the current position, but that's not the main thing it does).
</details>


<details><summary>Solution</summary>

```python
def patch_final_move_output(
    activation: Float[Tensor, "batch seq d_model"],
    hook: HookPoint,
    clean_cache: ActivationCache,
) -> Float[Tensor, "batch seq d_model"]:
    """
    Hook function which patches activations at the final sequence position.

    Note, we only need to patch in the final sequence position, because the prior moves in the clean
    and corrupted input are identical (and this is an autoregressive model).
    """
    activation[0, -1, :] = clean_cache[hook.name][0, -1, :]
    return activation


def get_act_patch_resid_pre(
    model: HookedTransformer,
    corrupted_input: Float[Tensor, "batch pos"],
    clean_cache: ActivationCache,
    patching_metric: Callable[[Float[Tensor, "batch seq d_model"]], Float[Tensor, ""]],
) -> Float[Tensor, "2 n_layers"]:
    """
    Returns an array of results corresponding to the results of patching at each (attn_out, mlp_out)
    for all layers in the model.
    """
    model.reset_hooks()
    results = t.zeros(2, model.cfg.n_layers, device=device, dtype=t.float32)
    hook_fn = partial(patch_final_move_output, clean_cache=clean_cache)

    for i, activation in enumerate(["attn_out", "mlp_out"]):
        for layer in tqdm(range(model.cfg.n_layers)):
            patched_logits = model.run_with_hooks(
                corrupted_input,
                fwd_hooks=[(get_act_name(activation, layer), hook_fn)],
            )
            results[i, layer] = patching_metric(patched_logits)

    return results
```
</details>

### Recap of this section

We did the following:

* Learned how **activation patching** worked.
* Constructed the following datasets for patching:
    * Clean distribution = unaltered game,
    * Corrupted distribution = game with a single move flipped (changing th legality of a square),
* Looked at the effect on patching at the **output of attention and MLP layers** to see which ones changed the output significantly.
    * We found a handful of the MLP layers, and the final attention layer, mattered.
        * Attention layers mostly not mattering was unsurprising, since attention's main job is to move around information rather than operate on it.
    * If we wanted, we could get more granular at this point, and explore which neurons in these layers had a significant effect.

# 3️⃣ Neuron Interpretability: A Deep Dive

> ##### Learning Objectives
>
> - Apply **direct logit attribution** to understand how a neuron's output weights directly effect predictions
> - Use SVD-based techniques to assess how much of a neuron's input/output behaviour is captured by some subspace
> - Use techniques like **max activating datasets** and **spectrum plots**, and understand their strengths and limitations

To practice neuron interpretability, let's take a deep dive into understanding this neuron - the techniques and code should transfer pretty well to any other neurons!

The spirit of this section is to practice doing various standard things that you could go and apply to another neuron in practice - I end it still being fairly confused, and with many dangling threads!

As above, we can use the probe weights to analyse its input weights. So we go in with the hypothesis that it detects `(C0==BLANK) & (D1==THEIRS) & (E2==MINE)`, and that it directly boosts the `C0` logit - this configuration means that `C0` must be legal, because it would flank `D1` along the diagonal!

## Direct logit attribution

As we studied briefly in the previous section, we can look at the **direct logit attribution** of a neuron's output weights - i.e. `w_out @ W_U`, to see if the neuron significantly affects the output logits directly. Note that this is purely weights based analysis, we're not yet looking at actual model activations.

In [113]:
layer = 5
neuron = 1393

# Get neuron output weights in unembedding basis
w_out = get_w_out(model, layer, neuron, normalize=False)
w_out_W_U_basis = w_out @ model.W_U[:, 1:]  # shape (60,)

# Turn into a (rows, cols) tensor, using indexing
w_out_W_U_basis_rearranged = t.zeros((8, 8), device=device)
w_out_W_U_basis_rearranged.flatten()[ALL_SQUARES] = w_out_W_U_basis

# Plot results
utils.plot_board_values(
    w_out_W_U_basis_rearranged,
    title=f"Cosine sim of neuron L{layer}N{neuron} with W<sub>U</sub> directions",
    width=450,
    height=380,
)

Here we immediately see that boosting `C0` is an important part of what happens (which fits with our earlier hypothesis about the diagonal pattern which this MLP was picking up on), but that it also boosts `D1`, which is surprising! (`D1` is filled in our hypothesis, so must be illegal!)

One hypothesis is that the unembed for `C0` and `D1` are highly aligned, so it can't easily boost one but not the other.

### Exercise - compare the unembeds for C0 and D1

> ```yaml
> Difficulty: 🔴🔴⚪⚪⚪
> Importance: 🔵🔵⚪⚪⚪
>
> You should spend up to 5-10 minutes on this exercise.
> ```

Is this hypothesis true? Test it by calculating the cosine similarity of the `C0` and `D1` unembeds.

Recall, you can use the `utils.label_to_id` helper function to convert a label like `"C0"` to an integer index (which you can use to index into the unembedding).

In [119]:
print(model.W_U.shape)

torch.Size([512, 61])


In [127]:
# YOUR CODE HERE - calculate cosine sim between unembeddings
c0_U = model.W_U[:, utils.label_to_id("C0")].detach()
c0_U /= c0_U.norm()

d1_U = model.W_U[:, utils.label_to_id("D1")].detach()
d1_U /= d1_U.norm()

cos_sim = c0_U @ d1_U
print("cos sim between C0 and D1: %.3f"%(cos_sim.item()))

cos sim between C0 and D1: -0.022


<details>
<summary>Answer (what you should get)</summary>


You should see that the cosine similarity is close to zero, i.e. they're basically orthogonal. So this hypothesis is false! What else do you think might be going on here?

</details>


<details><summary>Solution</summary>

```python
c0_U = model.W_U[:, utils.label_to_id("C0")].detach()
c0_U /= c0_U.norm()

d1_U = model.W_U[:, utils.label_to_id("D1")].detach()
d1_U /= d1_U.norm()

print(f"Cosine sim of C0 and D1 unembeds: {c0_U @ d1_U:.3f}")
```
</details>

To check that this is a *big* part of the neuron's output, lets look at the fraction of variance of the output captured by the $W_U$ subspace.

### Exercise - compute fraction of variance explained by unembedding subspace

> ```yaml
> Difficulty: 🔴🔴🔴⚪⚪
> Importance: 🔵🔵⚪⚪⚪
>
> You should spend up to 10-20 minutes on this exercise.
> This might be a bit harder to understand conceptually, without a linear algebra background.
> ```

Note - when we say "fraction of a vector's variance explained by a subspace", this means the fraction of the original vector's squared norm which is represented by its projection onto the subspace. So if we have a vector $v$ and a subspace $W$ represented by a matrix of orthogonal row vectors, then $v^T W$ are the components of the projection of $v$ onto $W$, and the fraction of $v$'s variance explained by the subspace $W$ is $\|v^T W\|^2 / \|v\|^2$.

If you're confused by this, then you should go to the [How much variance does the probe explain?](#how-much-variance-does-the-probe-explain) section, where we do a computation which is similar to (but more complicated than) to the one which you should do now.

*Gotcha - remember to remove the 0th vocab entry from $W_U$ before taking SVD, since this corresponds to the "pass" move which never comes up in our data.*

In [128]:
# YOUR CODE HERE - compute the variance frac of neuron output vector explained by unembedding subspace
w_out = get_w_out(model, layer, neuron, normalize=True)
U, S, Vh = t.svd(model.W_U[:, 1:])
print(f"Fraction of variance captured by W_U: {((w_out @ U).norm().item() ** 2):.4f}")

Fraction of variance captured by W_U: 0.2868


<details>
<summary>Solution (code and what you should get)</summary>

```python
w_out = get_w_out(model, layer, neuron, normalize=True)
U, S, Vh = t.svd(model.W_U[:, 1:])
print(f"Fraction of variance captured by W_U: {((w_out @ U).norm().item() ** 2):.4f}")
```

You should find about 1/4 of the variance is explained:

<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace">Fraction of variance captured by W_U: 0.2868</pre>

This is less than we might expect, and suggests that more might be going on here. Maybe the output of this neuron is used bu something else downstream before the unembedding is applied?

</details>

Another quick sanity check is just plotting what the neuron activations look like over the 50 games in our cache - we see that it only matters in a few games, and matters every other move (which makes sense for a feature denominated in terms of my vs their colours - this alternates every move as the colours swap!)

In [132]:
neuron_acts = focus_cache["post", layer, "mlp"][:, :, neuron]

fig = px.imshow(
    to_numpy(neuron_acts),
    title=f"L{layer}N{neuron} Activations over 50 games",
    labels={"x": "Move", "y": "Game"},
    color_continuous_scale="RdBu",
    color_continuous_midpoint=0.0,
    aspect="auto",
    width=900,
    height=400,
)
fig.show()

### Exercise - figure out what's going on with one of these games

> ```yaml
> Difficulty: 🔴🔴🔴⚪⚪
> Importance: 🔵🔵🔵🔵⚪
>
> You should spend up to 10-25 minutes on this exercise.
> This is a very valuable exercise for testing your ability to develop new hypotheses, not just writing code.
> ```

Take one of the games where this neuron fires (e.g. game 5 seems like a good example). Plot the state of the game using `plot_board_values` (you can take code from earlier in this notebook).

<details>
<summary>Answer (for game 5)</summary>

Our neuron fires on moves 10, 12, 14, 16, 18 and 20. By our theory, we expect to find that these are precisely the moves where `C0` is legal because it allows us to capture along the diagonal which contains `D1`.

```python
imshow(
    focus_states[5, :25],
    facet_col=0,
    facet_col_wrap=5,
    y=list("ABCDEFGH"),
    facet_labels=[f"Move {i}" for i in range(25)],
    title="First 16 moves of first game",
    color_continuous_scale="Greys",
    coloraxis_showscale=False,
    width=1000,
    height=1000,
)
```

This is exactly what we find. `C0` is a legal next move for the states labelled 10, 12, 14, 16 and 18. In all cases, it's white's move, and the legality is because `D1` is occupied by black and `E2` by white. Note that the `C0` square is legal in some other states (e.g. after move 22, when it's black's move), but we don't expect the neuron to fire in this case because the legality isn't because we can capture along the `C0-D1-E2` diagonal.
</details>

In [148]:
game_index = 5
m1 = 10
m2 = 17
moves = list(range(m1,m2,1))

# Plot board state
utils.plot_board_values(
    focus_states[game_index, moves],
    # title="Focus game states",
    width=1000,
    height=400,
    text=focus_legal_moves_annotation[game_index][m1:m2],
)

## Max Activating Datasets

Max activating datasets are a useful but also sometimes misleading tool we can use to better understand a neuron's activations. From my [Dynalist notes](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=pwjeUj-94p6EwwMO_41Kb3h1):

* **Max Activating Dataset Examples** aka **dataset examples** aka **max examples** are a simple technique for neuron interpretability. The model is run over a lot of data points, and we select the top K data points by how much they activate that neuron.
    * Sometimes there is a clear pattern in these inputs (eg they’re all pictures of boats), and this is (weak!) evidence that the neuron detects that pattern. Sometimes there are multiple clusters of inputs according to different patterns, which suggests that the neuron is **polysemantic** (i.e. it represents some combination of multiple different features).
    * This is a very simple and dumb technique, and has faced criticism, see e.g. [The Interpretability Illusion](https://arxiv.org/abs/2104.07143) which found that different datasets gave different sets of examples, each of which had a __different__ clear pattern.
    * See outputs of this for image models in [OpenAI Microscope](https://microscope.openai.com/) and language models in [Neuroscope](https://neuroscope.io/).

Doing this properly across many games is effort, but here we have cached outputs on 50 games (3000 moves), so we can look at the top 30 games there.

In [149]:
# Get top 30 games & plot them all
top_moves = neuron_acts > neuron_acts.quantile(0.99)
top_focus_states = focus_states[:, :-1][top_moves.cpu()]
top_focus_states_flip = focus_states_theirs_vs_mine[:, :-1][top_moves.cpu()]
utils.plot_board_values(
    top_focus_states,
    boards_per_row=10,
    board_titles=[f"{act=:.2f}" for act in neuron_acts[top_moves]],
    title=f"Top 30 moves for neuron L{layer}N{neuron}",
    width=1600,
    height=500,
)

# Plot heatmaps for how frequently any given square is mine/theirs/blank in those top 30
utils.plot_board_values(
    t.stack([top_focus_states_flip == 0, top_focus_states_flip == 1, top_focus_states_flip == 2]).float().mean(1),
    board_titles=["Blank", "Theirs", "Mine"],
    title=f"Aggregated top 30 moves for neuron L{layer}N{neuron}, in 'blank/mine/theirs' basis",
    width=800,
    height=380,
)

Note that the blank cells show up with high frequency - this is one of the kinds of things [you have to watch out for](https://arxiv.org/abs/2104.07143), since exactly how you've defined your dataset will affect the features of the max activating dataset you end up finding (and it might be the case that each dataset exhibits a clear but misleading pattern).

We mostly care about the "mine vs theirs" distinction here because it's the most interesting, so let's create a new tensor where "mine" is `-1` and "theirs" is `+1`, then taking a mean over this tensor will give us an idea of "mine vs theirs".

In [166]:
print(focus_states_theirs_vs_mine_pm1.shape)

torch.Size([50, 60, 8, 8])


In [150]:
focus_states_theirs_vs_mine_pm1 = t.zeros_like(focus_states_theirs_vs_mine, device=device)
focus_states_theirs_vs_mine_pm1[focus_states_theirs_vs_mine == 2] = 1
focus_states_theirs_vs_mine_pm1[focus_states_theirs_vs_mine == 1] = -1

board_state_at_top_moves = focus_states_theirs_vs_mine_pm1[:, :-1][top_moves].float().mean(0)
board_state_at_top_moves.shape

utils.plot_board_values(
    board_state_at_top_moves,
    title=f"Aggregated top 30 moves for neuron L{layer}N{neuron}<br>(1 = theirs, -1 = mine)",
    height=380,
    width=450,
)

### Exercise - investigate more neurons

> ```yaml
> Difficulty: 🔴🔴🔴⚪⚪
> Importance: 🔵🔵🔵🔵⚪
>
> You should spend up to 15-30 minutes on this exercise.
> This exercise involves applying the same techniques you used above to other neurons, in a batched way.
> ```

We see that, for all moves in the max activating dataset, `D1` is theirs and `E2` is mine. This is moderately strong evidence that our neuron is doing the thing we think it's doing.

Let's review the kinds of plots we've made during this section. There have been two kinds:

1. **Direct logit attribution plots**, where we calculated the cosine similarity of the corresponding neuron with the unembedding weights corresponding to each square.

<details>
<summary>Example</summary>

<img src="https://raw.githubusercontent.com/info-arena/ARENA_img/main/misc/plottwo.png" width="950">

</details>

2. **Max activating dataset plots**, where we get a heatmap of how frequently a square was mine / theirs over the dataset we're choosing (where that dataset is the max activating dataset for some particular neuron).

<details>
<summary>Example</summary>

<img src="https://raw.githubusercontent.com/info-arena/ARENA_img/main/misc/plotone.png" width="750">
</details>

Try and make both of these plots, but for **all the top 10 neurons in layer 5** (where, like before, we measure "top 10" by the standard deviation of the neurons' activations, over all games and moves from the data in `focus_cache`). You'll be able to copy and paste some code from earlier.

We've given you the code for making the plots. All you need to do is calculate the `output_weights_in_logit_basis` and `board_states` tensors, which should be batched versions of the tensors which were used to generate the plots above (the 0th axis should be the neuron index, i.e. the `[0, ...]`-th slice of each of these tensors should be the things we fed into our plotting functions earlier in this section).

Can you guess what any of these neurons are doing? Does it help if you also plot some of them in the logit basis, like you did for `L5N1393`?

In [156]:
print(model.W_U[:,1:].shape)

torch.Size([512, 60])


In [185]:
# YOUR CODE HERE - investigate the top 10 neurons by std dev of activations, see what you can find!
layer = 5
top_neurons = focus_cache["post", layer].std(dim=[0, 1]).argsort(descending=True)[:10]
index = t.arange(10)

l5_w_out = t.stack([get_w_out(model, layer, n.item(), normalize=False) for n in top_neurons], dim=0)

l5_w_out_W_U = l5_w_out @ model.W_U[:,1:] # [10, 60]
l5_w_out_W_U_basis = t.zeros((10, 8, 8), device=device)
for n in range(10):
    l5_w_out_W_U_basis[n,:,:].flatten()[ALL_SQUARES] = l5_w_out_W_U[n,:]

# Plot results
utils.plot_board_values(
    l5_w_out_W_U_basis,
    title="output weights of top 10 neurons in layer 5 in output logit basis",
    board_titles=[f"L{layer}N{n.item()}" for n in top_neurons],
    width=2000,
    height=380,
)


# aggregate top moves
l5_neuron_acts = focus_cache["post", layer, "mlp"][:, :, top_neurons]
board_state_at_top_moves = t.empty((10,8,8))
for n in range(10):
    l5_neuron_acts_n = l5_neuron_acts[:,:,n]
    top_moves = l5_neuron_acts_n > l5_neuron_acts_n.quantile(0.99)
    board_state_at_top_moves[n,:,:] = focus_states_theirs_vs_mine_pm1[:, :-1][top_moves].float().mean(0)

utils.plot_board_values(
    board_state_at_top_moves,
    title=f"Aggregated top moves for top 10 neurons in layer 5",
    board_titles = [f"L5N{n.item()}" for n in top_neurons],
    height=380,
    width=2000,
)

<details><summary>Solution</summary>

```python
layer = 5
top_neurons = focus_cache["post", layer].std(dim=[0, 1]).argsort(descending=True)[:10]
board_states = []
output_weights_in_logit_basis = []

for neuron in top_neurons:
    # Get output weights in logit basis
    w_out = get_w_out(model, layer, neuron, normalize=False)
    state = t.zeros(8, 8, device=device)
    state.flatten()[ALL_SQUARES] = w_out @ model.W_U[:, 1:]
    output_weights_in_logit_basis.append(state)

    # Get max activating dataset aggregations
    neuron_acts = focus_cache["post", 5, "mlp"][:, :, neuron]
    top_moves = neuron_acts > neuron_acts.quantile(0.99)
    board_state_at_top_moves = focus_states_theirs_vs_mine_pm1[:, :-1][top_moves].float().mean(0)
    board_states.append(board_state_at_top_moves)


output_weights_in_logit_basis = t.stack(output_weights_in_logit_basis)
board_states = t.stack(board_states)

utils.plot_board_values(
    output_weights_in_logit_basis,
    title=f"Output weights of top 10 neurons in layer {layer}, in the output logit basis",
    board_titles=[f"L{layer}N{n.item()}" for n in top_neurons],
    width=1600,
    height=360,
)
utils.plot_board_values(
    board_states,
    title=f"Aggregated top 30 moves for each top 10 neuron in layer {layer}",
    board_titles=[f"L{layer}N{n.item()}" for n in top_neurons],
    width=1600,
    height=360,
)
```
</details>

How do you interpret the results?

<details>
<summary>Answer (a few examples)</summary>

Like the previous neuron, some of these should be interpretable. Examples:

* `L5N1406` - if `D4` is theirs and `D5` is blank, then this boosts the logits for `D5`.
* `L5N1985` - if `F4` is blank, and the opponents' pieces are adjacent to it, then this boosts the logits for `F4`.
</details>

## Spectrum Plots

One of the best ways to validate a hypothesis about neurons is to use a **spectrum plot**, where we plot a histogram of the neuron activations across the full data distribution (or at least some random sample) and categorise each activation by whether it has the property we think the neuron is detecting. We can do a janky version of that with the neuron's activations on our 50 games (plotting each histogram group as percent of the group size, to normalise for most games not having the configuration hypothesised).

Fascinatingly, we see that our hypothesis did *not* fully capture the neuron - almost every high activation had the hypothesised configuration, but so did some low ones! I'm not sure what's going on here, but it reveals a weakness of max activating dataset examples - they don't let you notice when your hypothesis allows false positives!

<details>
<summary>Question - can you explain why some of the activations are negative?</summary>

Remember we're using `GELU`, which can be negative.
</details>

In [186]:
c0 = focus_states_theirs_vs_mine_pm1[:, :, 2, 0]
d1 = focus_states_theirs_vs_mine_pm1[:, :, 3, 1]
e2 = focus_states_theirs_vs_mine_pm1[:, :, 4, 2]

label = (c0 == 0) & (d1 == -1) & (e2 == 1)

neuron_acts = focus_cache["post", 5][:, :, 1393]


def make_spectrum_plot(neuron_acts: Float[Tensor, "batch"], label: Bool[Tensor, "batch"], **kwargs) -> None:
    """
    Generates a spectrum plot from the neuron activations and a set of labels.
    """
    px.histogram(
        pd.DataFrame({"acts": neuron_acts.tolist(), "label": label.tolist()}),
        x="acts",
        color="label",
        histnorm="percent",
        barmode="group",
        color_discrete_sequence=px.colors.qualitative.Bold,
        nbins=100,
        **kwargs,
    ).show()


make_spectrum_plot(
    neuron_acts.flatten(),
    label[:, :-1].flatten(),
    title="Spectrum plot for neuron L5N1393 testing C0==BLANK & D1==THEIRS & E2==MINE",
    width=1200,
    height=400,
)

### Exercise - investigate this spectrum plot

> ```yaml
> Difficulty: 🔴🔴🔴🔴🔴
> Importance: 🔵🔵🔵🔵⚪
>
> This is a very open-ended exercise, with no time estimates. You can come back to this at the end if you like.
> ```

Look at the moves with this configuration and low activations - what's going on there? Can you see any patterns in the board states? In the moves? What does the neuron activation look like over the course of the game?

### Exercise - make more spectrum plots

> ```yaml
> Difficulty: 🔴🔴🔴🔴⚪
> Importance: 🔵🔵🔵🔵⚪
>
> This is a very open-ended exercise, with no time estimates. You can come back to this at the end if you like.
> ```

Try to find another neuron with an interpretable attention pattern. Make a spectrum plot for it. What do you find?

### Recap of this section

In this section, we did the following:

* Used **direct logit attribution** to see how the output weights of neurons were affecting the output logits for each square.
    * Found `L5N1393` was boosting the logits for cell `C0`.
* Used **max activating datasets** to see which kinds of game states were causing certain neurons to fire strongly.
    * Found `L5N1393` was firing strongly when on the `(C0==BLANK) & (D1==THEIRS) & (E2==MINE)` pattern.
* Repeated these two plots for a bunch of other neurons, and found that lots of them were interpretable in similar ways (i.e. they all activated strongly on a particular pattern, and positively affected the logits for a square which would be legal to play in that pattern).
* Made a **spectrum plot**, and found that our explanation of the neuron wasn't the whole story (some game states with the given pattern *didn't* cause the neuron to fire).
    * This revealed a weakness with max activating datasets as a method of finding a full explanation of a neuron's behaviour.

# 4️⃣ Training a Probe

> ##### Learning Objectives
>
> - Learn how to set up and train a linear probe
> - See how to train multiple probes at once, and log their performance to Weights & Biases

In this final section, we'll return to the linear probe from earlier, but discuss how you might go about training it from scratch.

We won't be doing a full-scale training run here, instead we'll just look at a small example involving the `board_seqs_square_small.npy` datasets we've already used (the actual probe we've been using in these exercises was trained using a larger `board_seqs_int.pth` dataset).

Note - if you're an ARENA participant who did the material on [model training](https://arena-ch0-fundamentals.streamlit.app/[0.3]_ResNets) during the first week (or in the "transformer from scratch" material), this should all be familiar to you. I'd recommend doing that section before this one, unless you already have experience writing standard ML training loops.

One important thing to make clear: we're not actually training our transformer model here. If this was a standard training loop, we'd run our model in training mode and update its gradients in a way which reduces the cross entropy loss between its logit output and the true black/white/blank labels. Instead, we're **running our model in inference mode, caching its residual stream values, applying our probe to these values, and then updating the weights of our probe in a way which reduces the cross entropy loss between our probe's output and the true mine/theirs/blank labels.**

In [ ]:
utils.plot_board_values(
    focus_states[0, :16],
    boards_per_row=8,
    board_titles=[f"Move {i}" for i in range(1, 17)],
    title="First 16 moves of first game",
    width=1400,
    height=440,
)

Now, we'll create a **dataclass** to store our probe training args. This is a great way to keep all our variables in one place (and also it works well with VSCode's autocompletion features!). Also, note a cool feature of dataclasses - you can define attributes in terms of previous attributes (e.g. see the `length` attribute).

We've also included a `setup_linear_probe` method, which will give us a randomly initialized probe with appropriately normalized weights.

In [ ]:
@dataclass
class ProbeTrainingArgs:
    # Determine the activations we'll train the probe on
    layer: int = 6
    pos_start: int = 5
    pos_end: int = -5  # i.e. we slice [pos_start: model.n_ctx + pos_end]

    # Game state (options are blank/mine/theirs)
    options: int = 3
    rows: int = 8
    cols: int = 8

    # Standard training hyperparams
    epochs: int = 3
    num_games: int = 10_000

    # Hyperparams for optimizer
    batch_size: int = 32
    lr: float = 1e-3  # high LR for quick convergence in these exercises; you may want to reduce
    betas: tuple[float, float] = (0.9, 0.99)
    weight_decay: float = 0.01

    # Saving & logging
    use_wandb: bool = False
    wandb_project: str | None = "othellogpt-probe"
    wandb_name: str | None = None

    # Code to get randomly initialized probe
    def setup_linear_probe(self, model: HookedTransformer):
        linear_probe = t.randn(model.cfg.d_model, self.rows, self.cols, self.options, device=device) / np.sqrt(
            model.cfg.d_model
        )
        linear_probe.requires_grad = True
        return linear_probe

A reminder of what some of these mean:
* `modes` refers to "black to play/odd moves", "white to play/even moves", and "all moves". In the previous exercises, we only ever used "black to play" (this choice didn't really matter, since the model is detecting "my/their color" rather than "black/white").
* `options` (for our linear probe) refers to "empty", "black" and "white". After we've trained it, we'll create a version with "empty", "theirs" and "mine".

Now for our main block of code - we'll write a class for training our linear probe.

### Exercise - fill in the missing code below

> ```yaml
> Difficulty: 🔴🔴🔴🔴⚪
> Importance: 🔵🔵🔵⚪⚪
>
> You should spend up to 30-40 minutes on this exercise.
> There are several steps to this exercise, so after trying for some time you're recommended to look at the solution.
> ```

We've just left the `training_step` function incomplete, so that's the one you need to fill in. It should return the loss which you backpropagate on (you can see the code which actually performs the backprop algorithm below).

To make this exercise easier we're only having you train a single probe mode, on even moves (as a bonus exercise you can try training 3 modes at once - if you do this then remember to flip the labels for odd and even moves when you compute loss for the "even & odd" probe mode).

Your `training_step` function should:

- Use `model.run_with_cache` to get the cached residual stream values for the layer you're training on.
    - Tip: you can use the `stop_at_layer` argument to prevent unnecessary computation
    - Tip: you can also the `names_filter` argument to only get the activations you need (this can be a hook name or a function mapping hook names to bools)
    - Tip: remember to use inference mode here, since we're only running our model to get the activations to feed into our probe.
    - Tip: remember that the `games_id` tensor you're given has shape `(batch_size, 60)`, so you'll need to slice off the last game before passing into the model.
- Slice the activations appropriately (we have `args.pos_start` and `args.pos_end` telling you which positions to use, and also we're only using the even positions for now)
- Compute the probe output, by taking an inner product over the `d_model` dimension and summing over positions.
- Convert the probe output to logprobs, and get correct logprobs by indexing into it.
    - Tip: we're using even positions, so remember that we convert from board state to our probe basis using "mine=black, theirs=white".
    - Tip: this indexing will require taking code from earlier which computes board state from the game sequence. We recommend just using `get_board_states_and_legal_moves` and keeping the `state` object returned - remember that this state object has values 0, 1, 2 corresponding to options empty, white, black.
- Compute the loss, by summing over the row & column dimensions and averaging over the batch & seqpos dimensions (this is because the latter are data batch dimensions, but the former are effectively probe batch dimensions, since we're training multiple probes at once on each board square).
- Return the loss (this code is given to you).

Once you've finished implementing this function, you can run the code below. Your training loss should quickly drop below `64 * ln(3) ≈ 70` (which is the loss you'd get from uniform random guesses about what occupies each square). This loss should fall to around 10-20 by the end of training, if you use the default hyperparameters.

We recommend you set `use_wandb=False` while working on this code, until you're getting it running without errors.

In [ ]:
class LinearProbeTrainer:
    def __init__(self, model: HookedTransformer, args: ProbeTrainingArgs):
        self.model = model
        self.args = args
        self.linear_probe = args.setup_linear_probe(model)

    def training_step(self, indices: Int[Tensor, "n_games"]) -> Float[Tensor, ""]:
        # Use indices to slice our batch of games (remember, games_id = token IDs
        # from 1 to 60, and games_square = indices of squares in board)
        indices_cpu = indices.cpu()
        games_id = board_seqs_id[indices_cpu]  # shape [batch n_moves=60]
        games_square = board_seqs_square[indices_cpu]  # shape [batch n_moves=60]

        # Define seqpos slicing (note, we add n_ctx to pos_end to deal with the zero case)
        pos_start = self.args.pos_start
        pos_end = self.args.pos_end + self.model.cfg.n_ctx

        # YOUR CODE HERE - define loss

        if self.args.use_wandb:
            wandb.log(dict(loss=loss.item()), step=self.step)
        self.step += 1

        return loss

    def shuffle_training_indices(self):
        """
        Returns the tensors you'll use to index into the training data.
        """
        n_indices = self.args.num_games - (self.args.num_games % self.args.batch_size)
        full_train_indices = t.randperm(self.args.num_games)[:n_indices]
        full_train_indices = einops.rearrange(
            full_train_indices,
            "(batch_idx game_idx) -> batch_idx game_idx",
            game_idx=self.args.batch_size,
        )
        return full_train_indices

    def train(self):
        self.step = 0
        if self.args.use_wandb:
            wandb.init(project=self.args.wandb_project, name=self.args.wandb_name, config=self.args)

        optimizer = t.optim.AdamW(
            [self.linear_probe],
            lr=self.args.lr,
            betas=self.args.betas,
            weight_decay=self.args.weight_decay,
        )

        for epoch in range(self.args.epochs):
            print(f"Epoch {epoch + 1}/{self.args.epochs}")
            full_train_indices = self.shuffle_training_indices()
            progress_bar = tqdm(full_train_indices)
            for indices in progress_bar:
                loss = self.training_step(indices)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                progress_bar.set_description(f"Loss = {loss:.4f}")

        if self.args.use_wandb:
            wandb.finish()


t.set_grad_enabled(True)

args = ProbeTrainingArgs()
trainer = LinearProbeTrainer(model, args)
trainer.train()

<details><summary>Solution</summary>

```python
class LinearProbeTrainer:
    def __init__(self, model: HookedTransformer, args: ProbeTrainingArgs):
        self.model = model
        self.args = args
        self.linear_probe = args.setup_linear_probe(model)

    def training_step(self, indices: Int[Tensor, "n_games"]) -> Float[Tensor, ""]:
        # Use indices to slice our batch of games (remember, games_id = token IDs
        # from 1 to 60, and games_square = indices of squares in board)
        indices_cpu = indices.cpu()
        games_id = board_seqs_id[indices_cpu]  # shape [batch n_moves=60]
        games_square = board_seqs_square[indices_cpu]  # shape [batch n_moves=60]

        # Define seqpos slicing (note, we add n_ctx to pos_end to deal with the zero case)
        pos_start = self.args.pos_start
        pos_end = self.args.pos_end + self.model.cfg.n_ctx

        # Cache resid_post from all our games (ignoring the last one)
        with t.inference_mode():
            _, cache = model.run_with_cache(
                games_id[:, :-1].to(device),
                return_type=None,
                names_filter=lambda name: name.endswith("resid_post"),
            )

        # We slice from the first even index (on or after pos_start), since we're
        # just looking at predictions made after white has played a move.
        pos_start_even = pos_start + (pos_start % 2)
        seqpos_indices = np.arange(pos_start_even, pos_end, 2)
        resid_post = cache["resid_post", self.args.layer][:, seqpos_indices]

        # Get probe output, i.e. probe_logits[g, p, r, c] = the 3-vector of logit
        # predictions for what color is in square [r, c] AFTER the p-th move is
        # played in game g.
        probe_logits = einops.einsum(
            resid_post,
            self.linear_probe,
            "batch pos d_model, d_model rows cols options -> batch pos rows cols options",
        )
        probe_logprobs = probe_logits.log_softmax(-1)

        # Get the actual game state. The original state has {0: empty, 1: black, -1: white} and
        # we want our probe to be in the basis {0: empty, 1: theirs, 2: mine}. We're only training
        # on even moves i.e. black just played and mine = white, so we just need to map -1 -> 2.
        state = get_board_states_and_legal_moves(games_square)[0]  # shape [batch moves 8 8]
        state = state[:, seqpos_indices]  # shape [batch pos 8 8]
        state[state == -1] = 2

        # Index into probe logprobs to get the logprobs for correct board state, and then
        # return loss as the mean over games & posns, and sum over rows & cols (since each
        # row & col is effectively an independent probe).
        correct_probe_logprobs = eindex(probe_logprobs, state, "game pos row col [game pos row col]")
        loss = -einops.reduce(correct_probe_logprobs, "game pos row col -> row col", "mean").sum()

        if self.args.use_wandb:
            wandb.log(dict(loss=loss.item()), step=self.step)
        self.step += 1

        return loss

    def shuffle_training_indices(self):
        """
        Returns the tensors you'll use to index into the training data.
        """
        n_indices = self.args.num_games - (self.args.num_games % self.args.batch_size)
        full_train_indices = t.randperm(self.args.num_games)[:n_indices]
        full_train_indices = einops.rearrange(
            full_train_indices,
            "(batch_idx game_idx) -> batch_idx game_idx",
            game_idx=self.args.batch_size,
        )
        return full_train_indices

    def train(self):
        self.step = 0
        if self.args.use_wandb:
            wandb.init(project=self.args.wandb_project, name=self.args.wandb_name, config=self.args)

        optimizer = t.optim.AdamW(
            [self.linear_probe],
            lr=self.args.lr,
            betas=self.args.betas,
            weight_decay=self.args.weight_decay,
        )

        for epoch in range(self.args.epochs):
            print(f"Epoch {epoch + 1}/{self.args.epochs}")
            full_train_indices = self.shuffle_training_indices()
            progress_bar = tqdm(full_train_indices)
            for indices in progress_bar:
                loss = self.training_step(indices)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                progress_bar.set_description(f"Loss = {loss:.4f}")

        if self.args.use_wandb:
            wandb.finish()
```
</details>

Finally, let's make the same accuracy plot from before, and see how well it works. Note that we're not constructing a new probe by averaging the even and odd mode probes this time, we're just taking our single even mode probe and using it (there's a bonus exercise below where you can train all 3 modes at once).

In [ ]:
# Getting the probe's output, and then its predictions
probe_out = einops.einsum(
    focus_cache["resid_post", args.layer],
    trainer.linear_probe,
    "game move d_model, d_model row col options -> game move row col options",
)
probe_out_value = probe_out.argmax(dim=-1).cpu()

# See what the accuracy was in 3 cases: odd moves, even moves, and aggregate moves
is_correct = probe_out_value == focus_states_theirs_vs_mine[:, :-1]
accuracies_odd = einops.reduce(is_correct[:, 5:-5:2].float(), "game move row col -> row col", "mean")
accuracies_even = einops.reduce(is_correct[:, 6:-6:2].float(), "game move row col -> row col", "mean")
accuracies_all = einops.reduce(is_correct[:, 5:-5].float(), "game move row col -> row col", "mean")

utils.plot_board_values(
    1 - t.stack([accuracies_odd, accuracies_even, accuracies_all], dim=0),
    title="Average Error Rate of Linear Probe",
    board_titles=["Black to play", "White to play", "All Moves"],
    zmax=0.25,
    zmin=-0.25,
    height=400,
    width=900,
)

### Exercise - train all 3 modes at once (bonus)

> ```yaml
> Difficulty: 🔴🔴🔴🔴⚪
> Importance: 🔵⚪⚪⚪⚪
>
> You should spend up to 20-40 minutes on this exercise.
> ```

This exercise isn't super important or conceptually deep (and it can be very fiddly), but we include it for the sake of completionism. It's quite fiddly to get the indexing right here! What you should do is:

- Use a new probe training args class, where your linear probe has an extra `mode` dimension (we've given you code for this below).
- Rewrite your `training_step` function (in the new `LinearMultiProbeTrainer` class) to train all 3 modes in parallel.

We've also given you some sample code to run at the end, which will plot the accuracy of each of your 3 probes (as well as their ability to transfer to parities which they weren't trained on). You should see that your training loss for each of the 3 modes is about the same size as the total loss was in your previous probe, and they all reduce at approximately the same rate.

<details>
<summary>Help - I'm confused about how to rewrite the <code>training_step</code> function.</summary>

Previously, we just trained one linear probe on only the even moves. To break down the steps for this, we had to do the following:

- Compute the probe logprobs, with shape `(games, posn=59, rows=8, cols=8, options=3)`
- Remap the board state `state` from basis `{0: empty, 1: white, -1: black}` to `{0: empty, 1: theirs, 2: mine}`.
    - Since we were only using the even-valued positions, we could just slice them out and perform the mapping `{0, 1, -1} -> {0, 1, 2}`.
- Index into logprobs with the basis-mapped `state` to get the correct logprobs, which had shape `(games, posns, rows, cols)`.
- Compute loss = negative mean of correct logprobs (only including the even positions).

Now, we want to train 3 linear probes at once, one for even moves only, one for odd moves only, and one for both at once. This will look like:

- Compute the probe logprobs, with shape `(modes=3, games, posn=59, rows=8, cols=8, options=3)`
- Remap the board state `state` from basis `{0: empty, 1: white, -1: black}` to `{0: empty, 1: theirs, 2: mine}`.
    - Since we're using odd and even positions, this means mapping even positions with `{0, 1, -1} -> {0, 1, 2}` and odd positions with `{0, 1, -1} -> {0, 2, 1}`.
- Index into logprobs with the basis-mapped `state` to get the correct logprobs for each probe, which will have shape `(modes, games, posn, rows, cols)`.
- Compute loss = negative mean of correct logprobs (only including the even positions for our even probe, odd positions for our odd probe, and all positions for our all probe).

In other words it looks very similar, but we need to use the entire `state` tensor (with different basis mapping for odd and even moves) then make sure we're computing each loss over the correct set of sequence positions.

</details>

In [ ]:
@dataclass
class MultiProbeTrainingArgs(ProbeTrainingArgs):
    modes: int = 3  # even, odd, both (i.e. the data we train on)

    def setup_linear_probe(self, model: HookedTransformer):
        linear_probe = t.randn(
            self.modes,
            model.cfg.d_model,
            self.rows,
            self.cols,
            self.options,
            device=device,
        ) / np.sqrt(model.cfg.d_model)
        linear_probe.requires_grad = True
        return linear_probe


class LinearMultiProbeTrainer(LinearProbeTrainer):
    def training_step(self, indices: Int[Tensor, "n_games"]) -> Float[Tensor, ""]:
        indices_cpu = indices.cpu()
        games_id = board_seqs_id[indices_cpu]  # shape [batch n_moves=60]
        games_square = board_seqs_square[indices_cpu]  # shape [batch n_moves=60]

        pos_start = self.args.pos_start
        pos_end = self.args.pos_end + self.model.cfg.n_ctx

        # YOUR CODE HERE - define loss_even, loss_odd, loss_both
        loss = loss_even + loss_odd + loss_both

        if self.args.use_wandb:
            wandb.log(
                dict(
                    loss=loss.item(),
                    loss_even=loss_even.item(),
                    loss_odd=loss_odd.item(),
                    loss_both=loss_both.item(),
                ),
                step=self.step,
            )
        self.step += 1

        return loss


t.set_grad_enabled(True)

args = MultiProbeTrainingArgs(epochs=5)
trainer = LinearMultiProbeTrainer(model, args)
trainer.train()

In [ ]:
# Here, we test out each of our 3 probe modes (even / odd / both) on each of these 3 settings
# (even / odd / both). Hopefully we should see all 3 probes generalize!

probe_out = einops.einsum(
    focus_cache["resid_post", args.layer],
    trainer.linear_probe,
    "game move d_model, mode d_model row col options -> mode game move row col options",
)
probe_out_value = probe_out.argmax(dim=-1).cpu()  # mode game move row col

# For each mode, get the accuracy on even / odd / both
is_correct = probe_out_value == focus_states_theirs_vs_mine[:, :-1]  # mode game move row col
accuracies_even = einops.reduce(is_correct[:, 6:-6:2].float(), "mode game move row col -> mode row col", "mean")
accuracies_odd = einops.reduce(is_correct[:, 5:-5:2].float(), "mode game move row col -> mode row col", "mean")
accuracies_all = einops.reduce(is_correct[:, 5:-5].float(), "mode game move row col -> mode row col", "mean")

# Get all 3x3 accuracies, stacked over first dim
accuracies_stacked = t.concat([accuracies_even, accuracies_odd, accuracies_all], dim=0)

# Plot results!
board_titles = [
    f"{probe_mode} probe on {data_mode} data"
    for data_mode in ["even", "odd", "all"]
    for probe_mode in ["even", "odd", "both"]
]

utils.plot_board_values(
    1 - accuracies_stacked,
    title="Average Error Rate of Linear Probe",
    board_titles=board_titles,
    boards_per_row=3,
    zmax=0.25,
    zmin=-0.25,
    height=1000,
    width=900,
)

<details><summary>Solution</summary>

```python
@dataclass
class MultiProbeTrainingArgs(ProbeTrainingArgs):
    modes: int = 3  # even, odd, both (i.e. the data we train on)

    def setup_linear_probe(self, model: HookedTransformer):
        linear_probe = t.randn(
            self.modes,
            model.cfg.d_model,
            self.rows,
            self.cols,
            self.options,
            device=device,
        ) / np.sqrt(model.cfg.d_model)
        linear_probe.requires_grad = True
        return linear_probe


class LinearMultiProbeTrainer(LinearProbeTrainer):
    def training_step(self, indices: Int[Tensor, "n_games"]) -> Float[Tensor, ""]:
        indices_cpu = indices.cpu()
        games_id = board_seqs_id[indices_cpu]  # shape [batch n_moves=60]
        games_square = board_seqs_square[indices_cpu]  # shape [batch n_moves=60]

        pos_start = self.args.pos_start
        pos_end = self.args.pos_end + self.model.cfg.n_ctx

        # Cache resid_post from all our games (ignoring the last one)
        with t.inference_mode():
            _, cache = model.run_with_cache(
                games_id[:, :-1].to(device),
                return_type=None,
                names_filter=lambda name: name.endswith("resid_post"),
            )

        # We're training on all modes, so we slice all resid values in our range.
        resid_post = cache["resid_post", self.args.layer][:, pos_start:pos_end]

        # Get probe output, i.e. probe_logits[m, g, p, r, c] = the 3-vector of logit predictions from
        # mode-m probe, for what color is in square [r, c] AFTER the p-th move is played in game g.
        probe_logits = einops.einsum(
            resid_post,
            self.linear_probe,
            "game pos d_model, mode d_model rows cols options -> mode game pos rows cols options",
        )
        probe_logprobs = probe_logits.log_softmax(-1)

        # Get the actual game state. The original state has {0: empty, 1: black, -1: white} and
        # we want our probe to be in the basis {0: empty, 1: theirs, 2: mine}. For even moves,
        # mine = white, so we map -1 -> 2. For odd moves, mine = black, so we map {1, -1} -> {2, 1}.
        state = get_board_states_and_legal_moves(games_square)[0]
        state[:, ::2][state[:, ::2] == -1] = 2
        state[:, 1::2][state[:, 1::2] == 1] = 2
        state[:, 1::2][state[:, 1::2] == -1] = 1
        state = state[:, pos_start:pos_end]

        # Index into the probe logprobs with the correct indices (note, each of our 3 probe modes
        # gives us a different tensor of logprobs).
        correct_probe_logprobs = eindex(
            probe_logprobs,
            state,
            "mode game pos row col [game pos row col]",  # -> shape [mode game pos row col]
        )
        # Get the logprobs we'll be using for our 3 different probes. Remember that for the even
        # and odd probes we need to take only the even/odd moves respectively (and also that we've
        # already sliced logprobs from pos_start: pos_end).
        pos_start_even, pos_start_odd = (0, 1) if pos_start % 2 == 0 else (1, 0)
        even_probe_logprobs = correct_probe_logprobs[0, pos_start_even::2]
        odd_probe_logprobs = correct_probe_logprobs[1, pos_start_odd::2]
        both_probe_logprobs = correct_probe_logprobs[2]
        # Get our 3 different loss functions
        loss_even = -einops.reduce(even_probe_logprobs, "game pos row col -> row col", "mean").sum()
        loss_odd = -einops.reduce(odd_probe_logprobs, "game pos row col -> row col", "mean").sum()
        loss_both = -einops.reduce(both_probe_logprobs, "game pos row col -> row col", "mean").sum()
        # We backprop on the sum of all 3 losses
        loss = loss_even + loss_odd + loss_both

        if self.args.use_wandb:
            wandb.log(
                dict(
                    loss=loss.item(),
                    loss_even=loss_even.item(),
                    loss_odd=loss_odd.item(),
                    loss_both=loss_both.item(),
                ),
                step=self.step,
            )
        self.step += 1

        return loss
```
</details>

As a bonus exercise, you can try the following:

- Add an evaluation loop to the training code - this is helpful if you don't want to wait until the end of training to look at your probe classification accuracy!
- Make a hybrid probe by averaging over your even and odd mode probes (like the way we made probes from the provided probe in earlier exercises). Is this probe's accuracy higher than either the even or odd mode probe? Is it higher than the probe trained on both even and odd moves?

# ☆ Bonus

> Note - this was taken directly from the [second post in the OthelloGPT sequence](https://www.lesswrong.com/s/nhGNHyJHbrofpPbRG/p/qgK7smTvJ4DB8rZ6h#A_Transformer_Circuit_Laboratory); if you prefer then you can just read that instead. Scroll to the end to read some suggested replications for the final week.

_This is the second in a three post sequence about interpreting Othello-GPT. See [the first post](https://neelnanda.io/othello) for context._

_This post covers future directions I'm excited to see work on, why I care about them, and advice to get started. Each section is self-contained, feel free to skip around._

_Look up unfamiliar terms [here](https://neelnanda.io/glossary)_

The above sections leave me (and hopefully you!) pretty convinced that I've found something real and dissolved the mystery of whether there's a linear vs non-linear representation. But I think there's a lot of exciting mysteries left to uncover in Othello-GPT, _and_ that doing so may be a promising way to get better at reverse-engineering LLMs (the goal I actually care about). In the following sections, I try to:

*   Justify [why I think further work on Othello-GPT is interesting](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#Why_and_when_to_work_on_toy_models)
    *   (Note that my research goal here is to get better at transformer mech interp, [not to specifically understand emergent world models better](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#This_is_not_about_world_models))
*   Discuss how this unlocks [finding modular circuits](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#Finding_Modular_Circuits), and some preliminary results
    *   Rather than purely studying circuits mapping input tokens to output logits (like basically all prior transformer circuits work), using the probe we can study circuits mapping the input tokens to the world model, and the world model to the output logits - the difference between thinking of a program as a massive block of code vs being split into functions and modules.
    *   If we want to reverse-engineer large models, I think we need to get good at this!
*   Discuss how we can [interpret Othello-GPT's neurons](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#Neuron_Interpretability_and_Studying_Superposition) - we're very bad at interpreting transformer MLP neurons, and I think that Othello-GPT's are simple enough to be tractable yet complex enough to teach us something!
*   Discuss how, more broadly, [Othello-GPT can act as a laboratory](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#A_Transformer_Circuit_Laboratory) to get data on many other questions in transformer circuits - it's simple enough to have a ground truth, yet complex enough to be interesting

My hope is that some people reading this are interested enough to actually try working on these problems, and I end this section with [advice on where to start](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#Where_to_start_).

## Why and when to work on toy models

_This is a long and rambly section about my research philosophy of mech interp, and you should feel free to move on to [the next section](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#Finding_Modular_Circuits) if that's not your jam_

At first glance, playing legal moves in Othello (not even playing _good_ moves!) has nothing to do with language models, and I think this is a strong claim worth justifying. Can working on toy tasks like Othello-GPT really help us to reverse-engineer LLMs like GPT-4? I'm not sure! But I think it's a plausible bet worth making.

To walk through my reasoning, it's worth first thinking on what's holding us back - why haven't we already reverse-engineered the most capable models out there? I'd personally point to a few key factors (though note that this is my personal hot take, is not comprehensive, and I'm sure other researchers have their own views!):

*   **Conceptual frameworks**: To reverse-engineer a transformer, you need to know how to think like a transformer. Questions like: What kinds of algorithms is it natural for a transformer to represent, and how? Are features and circuits the right way to think about it? Is it even reasonable to expect that reverse-engineering is possible? How can we tell if a hypothesis or technique is principled vs hopelessly confused? What does it even mean to have truly identified a feature or circuit?
    *   I personally thought [A Mathematical Framework](https://transformer-circuits.pub/2021/framework/index.html) significantly clarified my conceptual frameworks for transformer circuits!
    *   This blog post is fundamentally motivated by forming better conceptual frameworks - do models form linear representations?
*   **Practical Knowledge/Techniques**: Understanding models is hard, and being able to do this in practice is hard. Getting better at this both looks like forming a better toolkit of techniques that help us form true beliefs about models, and also just having a bunch of practical experience with finding circuits and refining the tools - can we find any cases where they break? How can we best interpret the results?
    *   A concrete way this is hard is that models contain many circuits, each of which only activates on certain inputs. To identify a circuit we must first identify where it is and what it does, out of the morass! Activation patching (used in ROME, Interpretability in the Wild and refined with causal scrubbing) is an important innovation here.
*   **Understanding MLP Layers**: 2/3 of the parameters in transformers are in MLP layers, which process the information collected at each token position. We're pretty bad understanding them, and getting better at this is vital!
    *   We think these layers represent features as directions in space, and if each neuron represents a single feature, we're pretty good! But in practice this seems to be false, because of the poorly understood phenomena of superposition and polysemanticity
    *   [Toy Models of Superposition](https://transformer-circuits.pub/2022/toy_model/index.html) helped clarify my conceptual frameworks re superposition, but there's still a lot more to de-confuse! And a _lot_ of work to do to form the techniques to deal with it in practice. I'm still not aware of a single satisfying example of really understanding a circuit involving MLPs in a language model
*   **Scalability**: LLMs are _big_, and getting bigger all the time. Even if we solve all of the above in eg four layer transformers, this could easily involve some very ad-hoc and labour intensive techniques. Will this transfer to models far larger? And how well do the conceptual frameworks we form transformer - do they just break on models that are much more complex?
    *   This often overlaps with forming techniques (eg, causal scrubbing is an automated algorithm with the potential to scale, modulo figuring out many efficiency and implementation details). But broadly I don't see much work on this publicly, and would be excited to see more - in particular, checking how well our conceptual frameworks transfer, and whether all the work on small models is a bit of a waste of time!
        *   My personal hot take is that I'm more concerned about never getting _really_ good at interpreting a four layer model, than about scaling _if_ we're really good at four layer models - both because I just feel pretty confused about even small models, and because taking understood yet labour-intensive techniques and making them faster and more automatable seems hard but doable (especially with near-AGI systems!). But this is a complex empirical question and I could easily be wrong.

Within this worldview, what should our research goals be? Fundamentally, I'm an empiricist - models are hard and confusing, it's easy to trick yourself, and often intuitions can mislead. The core thing of any research project is getting feedback from reality, and using it to form true beliefs about models. This can either look like forming explicit hypotheses and testing them, or exploring a model and seeing what you stumble upon, but the fundamental question is whether you have the potential to be surprised and to get feedback from reality.

This means that any project is a trade-off between tractability and relevance to the end goal. Studying toy, algorithmic models is a double edged sword. They can be very tractable: they're clean and algorithmic which incentivises clean circuits, there's an available ground truth for what the model _should_ be doing, and they're often in a simple and nicely constrained domain. But it's extremely easy for them to cease to be relevant to real LLMs and become a nerd-snipe. (Eg, I personally spent a while working on grokking, and while this was very fun, I think it's just not very relevant to LLMs)

It's pretty hard to do research by constantly checking whether you're being nerd-sniped, and to me there are two natural solutions:

*   (1) To pick a concrete question you care about in language models, and to set out to specifically answer that, in a toy model that you're confident is a good proxy for that question
    *   Eg [Toy Models of Superposition](https://transformer-circuits.pub/2022/toy_model/index.html) built a pretty good toy model of residual stream superposition
*   (2) To pick a toy model that's a good enough proxy for LLMs _in general_, and just try hard to get as much traction on reverse-engineering that model as you can.
    *   Eg [A Mathematical Framework](https://transformer-circuits.pub/2021/framework/index.html) - I think that "train a model exactly like an LLM, but with only 1 or 2 layers" is pretty good as proxies go, though not perfect.

To me, working on Othello-GPT is essentially a bet on (2), that there in gneeral some are underlying principles of transformers and how they learn circuits, and that the way they manifest in Othello-GPT can teach us things about real models. This is definitely wrong in _some_ ways (I don't expect the specific circuits we find to be in GPT-3!), and it's plausible this is wrong in enough ways to be not worth working on, but I think it seems plausible enough to be a worthwhile research direction. My high-level take is just "I think this is a good enough proxy about LLMs that studying it hard will teach us generally useful things".

There's a bunch of key disanalogies to be careful of! Othello is fundamentally not the same task as language: Othello is a much simpler task, there's only 60 moves, there's a rigid and clearly defined syntax with correct and incorrect answers (not a continuous mess), the relevant info about moves so far can be _fully_ captured by the current board state, and generally many sub-tasks in language will not apply.

But it's also surprisingly analogous, at least by the standards of toy models! Most obviously, it's a transformer trained to predict the next token! But the task is also much more complex than eg modular addition, and it has to do it in weird ways! The way I'd code Othello is by doing it recursively - find the board state at move n and use it to get the state at move n+1. But transformers can't do this, they need to do things with a fixed number of serial steps but with a lot of computation in parallel (ie, at every move it must simultaneously compute the board state _at that move_ in parallel) - it's not obvious to me how to do this, and I expect that the way it's encoded will teach me a lot about how to represent certain kinds of algorithms in transformers. And it needs to be solving a bunch of sub-tasks that interact in weird ways (eg, a piece can be taken multiple times in each of four different directions), computing and remembering a lot of information, and generally forming coherent circuits.

In the next few sections I'll argue for how finding modular circuits can help build **practical knowledge and techniques**, what we could learn from **understanding its MLPs**, and more broadly how it could act as a laboratory for forming better **conceptual frameworks** (it's clearly not a good way to study scalability lol)

### This is not about world models

A high-level clarification: Though the focus of the original paper was on understanding how LLMs can form emergent world models, this is _not_ why I am arguing for these research directions. My interpretation of the original paper was that it was strong evidence for the fact that it's _possible_ for "predict the next token" models to form world emergent models, despite never having explicit access to the ground truth of the world/board state. I personally was already convinced that this was possible, but think the authors did great work that showed this convincingly and well (and I am even more convinced after my follow-up!), and that there's not much more to say on the "is this possible" question.

There's many interesting questions about whether these happen _in practice_ in LLMs and what this might look like and how to interpret it - my personal guess is that they do _sometimes_, but are pretty expensive (in terms of parameters and residual stream bandwidth) and only form when it's high value for reducing next token loss and the model is big enough to afford it. Further, there's often much cheaper hacks, eg, BingChat doesn't need to have formed an explicit chess board model to be decent at playing legal moves in chess! Probably not even for reasonably good legal play: the chess board state is _way_ easier than Othello, pieces can't even change colour! And you can get away with an implicit rather than explicit world model that just computes the relevant features from the context, eg to see where to a move a piece from, just look up the most recent point where that piece was played and look at the position it was moved to.

But Othello is very disanalogous to language here - playing legal moves in Othello has a single, perfectly sufficient world model that I can easily code up (though not quite in four transformer layers!), and which is incredibly useful for answering the underlying task! Naively, Othello-GPT roughly seems to be spending 128 of its 512 residual stream dimensions of this model, which is very expensive (though it's probably using superposition). So while it's a proof of concept that world models are possible, I don't think the finer details here tell us much about whether these world models actually happen in real LLMs. This seems best studied by actually looking at language models, and [I think there's many exciting questions here](/posts/XNjRwEX9kxbpzWFWd/200-cop-in-mi-looking-for-circuits-in-the-wild)! (eg doing mech interp on [Patel et al's work](https://openreview.net/pdf?id=gJcEM8sxHK)) The point of my investigation was more to refine our conceptual frameworks for thinking about models/transformers, and the goal of these proposed directions is to push forward transformer mech interp in general.

## Finding Modular Circuits

Basically all prior work on circuits (eg, [induction heads](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=_Jzi6YHRHKP1JziwdE02qdYZ), [indirect object identification](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=iWsV3s5Kdd2ca3zNgXr5UPHa), [the docstring circuit](/posts/u6KXXmKFbXfWzoAXn/a-circuit-for-python-docstrings-in-a-4-layer-attention-only), and [modular addition](https://arxiv.org/pdf/2301.05217.pdf)) have been on what I call **end-to-end** circuits. We take some model behaviour that maps certain inputs to certain outputs (eg the input of text with repetition, and the output of logits correctly predicting the repetition), and analyse the circuit going from the inputs to the outputs.

This makes sense as a place to start! The inputs and outputs are inherently interpretable, and the most obvious thing to care about. But it stands in contrast to much of the image circuits work, that identified neurons representing interpretable features ([like curves](https://distill.pub/2020/circuits/curve-circuits)) and studied how they were computed and how these were used to computed more sophisticated features (like [car wheels -> cars](https://distill.pub/2020/circuits/zoom-in)). Let's consider the analogy of mech interp to reverse-engineering a compiled program binary to source code. End-to-end circuits are like thinking of the source code as a single massive block of code, and identifying which sections we can ignore.

But a natural thing to aim for is to find **variables**, corresponding to interpretable activations within the network that correspond to **features**, some property of the input. The linear representation hypothesis says that these should be directions in activation space. It's not guaranteed that LLMs _are_ modular in the sense of forming interpretable intermediate features, but this seems implied by exiasting work, eg in the residual stream (often studied with probes), or in the MLP layers ([possibly as interpretable neurons](https://transformer-circuits.pub/2022/solu/index.html#section-6-3)). If we _can_ find interpretable variables, then the reverse-engineering task becomes much easier - we can now separately analyse the circuits that form the feature(s) from the inputs or earlier features, and the circuits that use the feature(s) to compute the output logits or more complex feature.

I call a circuit which starts or ends at some intermediate activation a **modular circuit** (in contrast to end-to-end circuits). These will likely differ in two key ways from end-to-end circuits:

*   They will likely be **shallower**, ie involving fewer layers of composition, because they're not end-to-end. Ideally we'd be able to eg analyse a single neuron or head in isolation.
    *   And hopefully easier to find!
*   They will be **composable** - rather than needing to understand a full end-to-end circuit, we can understand different modular circuits in isolation, and need only understand the input and output features of each circuit, not the circuits that computed them.
    *   Hopefully this also makes it easier to predict model behaviour off distribution, by analysing how interpretable units may compose in unexpected ways!

I think this is just obviously a thing we're going to need to get good at to have a shot at real frontier models! Modular circuits mean that we can both re-use our work from finding circuits before, and hopefully have many fewer levels of composition. But they introduce a new challenge - how do we find exactly what direction corresponds to the feature output by the first circuit, ie the **interface** between the two circuits? I see two natural ways of doing this:

*   Exploiting a privileged basis - finding interpretable neurons or attention patterns (if this can be thought of as a feature?) and using these as our interpretable foothold.
    *   This is great if it works, but superposition means this likely won't be enough.
*   Using probes to find an interpretable foothold in the residual stream or other activations - rather than assuming there's a basis direction, we learn the correct direction
    *   This seems the only kind of approach that's robust to superposition, and there's [a lot of existing academic work](https://arxiv.org/abs/2102.12452) to build upon!
    *   But this introduces new challenges - rather than analysing discrete units, it's now crucial to find the _right_ direction and easy to have errors. It seems hard to produce composable circuits if we can't find the right interface.

So what does any of this have to do with Othello-GPT? I think we'll learn a lot by practicing finding modular circuits in Othello-GPT. Othello-GPT has a world model - clear evidence of spontaneous modularity - and our linear probe tells us where it is in the residual stream. And this can be intervened upon - so we know there are downstream circuits that use it. This makes it a great case study! By about layer 4, of the 512 dimensions of the residual stream, we have 64 directions corresponding to which cell has "my colour" and 60 directions corresponding to which cells are blank (the 4 center cells are never blank). This means we can get significant traction on what _any_ circuit is reading or writing from the residual stream.

This is an attempt to get at the "**practical knowledge/techniques**" part of [my breakdown of mech interp bottlenecks](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#Why_and_when_to_work_on_toy_models) - Othello-GPT is a highly imperfect model of LLMs, but I expect finding modular circuits here to be highly tractable and to tell us a lot. Othello-GPT cares a lot about the world model - the input format of a sequence of moves is hard and messy to understand, while "is this move legal" can be answered purely from the board state. So the model will likely devote significant resources to computing board state, forming fairly clean circuits. Yet I still don't fully know how to do it, and I expect it to be hard enough to expose a bunch of the underlying practical and conceptual issues and to teach us useful things about doing this in LLMs.

**Gnarly conceptual issues**:

*   How to find the _right_ directions with a probe. Ie the correct interface between world-model-computing circuits and world-model-using circuits, such that we can think of the two independently. I see two main issues:
    *   Finding _all_ of the right direction - a probe with cosine sim of 0.7 to the "true" direction might work totally fine
        *   In particular, can we stop the probe from picking up on features that are constant in this context? Eg "is cell B6 my colour" is only relevant if "is cell B6 blank" is False, so there's naively no reason for the probe to be orthogonal to it.
    *   Ignoring features that correlate but are not causally linked - the corner cell can only be non-blank if at least one of the three neighbouring cells are, so the "is corner blank" direction _should_ overlap with these.
        *   But my intuition is that the model is learning a _causal_ world model, not correlational - if you want to do complex computations it's useful to explicitly distinguish between "is corner blank" as a thing to compute and use downstream, and all the other features. Rather than picking up on statistical correlations in the data.
*   If we find interpretable directions in the residual stream that are not orthogonal, how do we distinguish between "the model genuinely wants them to overlap" vs "this is just interference from superposition"?
    *   Eg, the model should want "is cell A4 blank" to have positive cosine sim with the unembed for the "A4 is legal" logit - non-blank cells are never legal!
*   The world model doesn't seem to be fully computed by layer X and only used in layer X+1 onwards - you sometimes need to intervene before layer 4, and sometimes the calculation hasn't finished before layer 5. How can we deal with overlapping layers? Is there a clean switchover layer _per cell_ that we can calculate separately?
*   How can we distinguish between two features having non-zero dot product because of noise/superposition, vs because they are correlated and the model is using one to compute the other.

**Questions I want answered**:

*   How _can_ we find the true probe directions, in a robust and principled way? Ideas:
    *   Use high weight decay to get rid of irrelevant directions. SGD (maybe with momentum) may be cleaner than AdamW here
    *   Use more complex techniques than logistic regression, like [amnesiac probing](https://arxiv.org/abs/2006.00995) (I found [Eleuther's Tuned Lens](https://arxiv.org/pdf/2303.08112.pdf) paper a useful review)
    *   Find the directions that work best for causal interventions instead.
    *   Maybe use the janky probe directions to try to find the heads and neurons that compute the world model, and use the fact that these are a privileged-ish basis to refine our understanding of the probe directions - if they never contribute to some component of the probe, probably that component shouldn't be there!
    *   Maybe implicitly assume that the probe directions should form an orthogonal set
    *   Maybe train a probe, then train a second probe on the residual stream component orthogonal to the first probe. Keep going until your accuracy sucks, and then take some kind of weighted average of the residual stream.
*   How is the blank world model computed?
    *   This _should_ be really easy - a cell is blank iff it has never been played, so you can just have an attention head that looks at previous moves. Maybe it's done after the layer 0 attention!
    *   This is trivial with an attention head per cell, but probably the model wants to be more efficient. What does this look like?
        *   Eg it might have a single attention head look at _all_ previous moves with uniform attention. This will get all of the information, but at magnitude `1/current_move`, maybe it has the MLP0 layer sharpen this to have constant magnitude?
    *   Meta question: What's a principled way to find the "is blank" direction here? The problem is one of converting a three-way classifier (blank vs my vs their) to a binary classifier that can be summarised with a single direction. I'm currently taking `blank - (my + their)/2`, but this is a janky approach
*   How is the "my vs their" world model computed?
    *   This seems like where the actual meat of the problem is!
        *   Consider games where
*   Which techniques work well here? My money is on activation patching and direct logit attribution being the main place to start, see [activation patching demoed in the accompanying notebook](https://neelnanda.io/othello-notebook).
    *   I'd love for someone to try out [attribution patching](https://www.neelnanda.io/mechanistic-interpretability/attribution-patching) here!
    *   By activation patching, I both mean resample ablations (patching a corrupted activation into a clean run to see which activations are vs aren't necessary) and causal tracing (patching a clean activation into a corrupted run to see which activations contain sufficient information to get the task right)

### Preliminary Results On Modular Circuits

The point of this section is to outline exciting directions of future work, but as a proof of concept I've done some preliminary poking around. The meta-level point that makes me excited about this is that linear probes are _really_ nice objects for interpretability. Fundamentally, transformers are made of linear algebra! Every component (layer, head and neuron) reads its input from the residual stream with a linear map, and writes it output by adding it to the residual stream, which is a _really_ nice structure.

**Probing across layers**: One way this is nice is that we can immediately get a foothold into understanding how the world model is computed. The residual stream is the sum of the embeddings and the output of every previous head and neuron. So when we apply a linear map like our probe, we can also break this down into a direct contribution from each previous head and neuron.

This is the same key idea as direct logit attribution, but now our projection is onto a probe direction rather than the unembed direction for a specific next token. This means we can immediately zoom in to the step of the circuit immediately before the probe, and see which components matter for each cell!

As an example, let's look at move 20 in this game:

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/heamef6n9xzedn5lkd1a" width="300">

The probe can perfectly predict the board state by layer 4

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/cozpptrltfdnolvyjvfo" width="500">

We can now look at how much the output of each attention and each MLP layer contributed to this (concretely we take the output of each attention and each MLP layer on move 30, and project them onto the is\_blank direction and the is\_mine direction for each cell, and plot this as a heatmap - [check the accompanying notebook for details](https://neelnanda.io/othello-notebook)). The MLP layer contributions to whether a cell has my or their colour is particularly interesting - we can see that it normally does nothing, but has a strong effect on the central stripe of cells that were just taken by the opponent - plausibly MLPs calculate when a cell is taken, and attention aggregates this? I'd love to see if there are specific neurons involve.

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/tqfl3q5mzwcbycxpcfi5" width="700">

**Reading Off Neuron Weights**: Another great thing about a linear probe is that it gives us a meaningful set of directions and subspace in the residual stream (beyond that given by the embedding and unembedding). This means that we can take any component's input or output weights, and project them onto the probe directions to see how that component reads to or writes from the probe's subspace - from this we can often just read off what's going on!

The probe intervention works best between layer 4 and layer 5, so we might hypothesise that some neurons in layer 5 are reading from the probe's subspace - we can check by taking the cosine sim of the neuron's input vector and the probe's directions to see how it responds to each, [see the accompanying notebook for details](https://neelnanda.io/othello-notebook). Here's neuron L5N1393 which seems to mostly represent C0==BLANK & D1==THEIRS & E2==MINE (cherry-picked for reasons unrelated to the probe, discussed more in post 3). Reading the figure: Blue = THEIRS, Red=MINE, White can be either blank or 50-50 mine vs their's, so can't be read easily.

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/egyxbgsfnmipmzunhtsj" width="600">

Here's the neurons with the largest standard deviation of activation in layer 3 (a pretty arbitrary way of choosing some that might be interesting) - when we take the cosine sim of the _output_ weights of these and the my colour probe, we see some that are pretty striking (though note that this is only a 0.3 cosine sim, so other stuff may be going on!)

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/og3g5rxemalzsggr9qpd" width="800">

Note that this is a deliberately janky analysis - eg, I'm not ensuring that the probe directions are orthogonal so I may double count, and I'm not looking for other residual stream features. You can track how reasonable this approach by tracking what fraction of the neuron's input is explained by the probe's subspaces, which is 64% in this case (these could otherwise be entirely spurious numbers!).

I go into neuron interpretability in more detail in the next section, but I think this technique is exciting in combination with what I discuss there, because it provides another somewhat uncorrelated technique - if many janky techniques give the same explanation about a neuron, it's probably legit!

## Neuron Interpretability and Studying Superposition

As argued earlier, I think that the current biggest open problem in transformer mech interp is understanding the MLP layers of transformers. These represent over 2/3 of the parameters in models, but we've had much more traction understanding attention-focused circuits. I'm not aware of a single public example of what I'd consider a well-understood circuit involving transformer MLP layers (beyond possibly my work on modular addition in a one layer transformer, but that's cheating). There are tantalising hints about the circuits they're used in in eg [SoLU](https://transformer-circuits.pub/2022/solu/index.html) and [ROME](https://rome.baulab.info/), but I broadly still feel confused re what is mechanistically going on. I think this is a thing we obviously need to make progress on as a field! And I think we'll learn useful things from trying to understand Othello-GPT's MLP layers!

What could progress on understanding MLPs in general look like? I think that we both need to get practice just studying MLP layers, and that we need to form clearer conceptual frameworks. A lot of our intuitions about transformer neurons come from image models, where neurons seem to (mostly?) represent features, have ReLU activations, and seem to be doing fairly discrete kinds of logic, eg "if car wheel present and car body present and car window present (in the right places) -> it's a car".

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/ze4jai5ioub31ovgewdj" width="800">

Transformers are different in a bunch of ways - there's attention layers, there's a residual stream (with significantly smaller dimension than the number of neurons in each layer!), and smoother and weirder GELU activations. Most importantly, polysemanticity seem to be a much bigger deal - single neurons often represent multiple features rather than a feature per neuron - and we think this is because models are using superposition - they represent features as linear combinations of neurons and use this to compress in more features than they have dimensions. This was argued for pretty convincingly in [Toy Models of Superposition](https://transformer-circuits.pub/2022/toy_model/index.html), but their insights were derived from a toy model, which can easily be misleading. I'm not aware of any work so far exhibiting superposition or properly testing the predictions of that paper in a real model. I expect some ideas will transfer but some will break, and that I'll learn a lot from seeing which is which!

Othello-GPT is far from a real language model, but I expect that understanding its MLP layers would teach me a bunch of things about how transformer MLP layers work in general. The model needs to compress a fairly complex and wide-ranging set of features and computation into just eight layers, and the details of _how_ it does this will hopefully expose some principles about what is and is not natural for a transformer to express in MLP neurons.

What would progress here look like? My high-level take is that a solid strategy is just going out, looking for interesting neurons, and trying to understand them deeply - no grander purpose or high-level questions about the model needed. I'd start with similar goals as I gave in [the previous section](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#Finding_Modular_Circuits) - look for the neurons that are used to compute the probe, and directly used by the probe. I also outline [some further preliminary results](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#Preliminary_Results_On_Modular_Circuits) that may serve as inspiration.

I've learned a lot from case studies looking deeply at concrete case studies of circuits in models: [Interpretability in the Wild](https://arxiv.org/abs/2211.00593) found backup heads (that took over when earlier heads were ablated) and negative heads (that systematically boosted _incorrect_ solutions), and [the docstring circuit](/posts/u6KXXmKFbXfWzoAXn/a-circuit-for-python-docstrings-in-a-4-layer-attention-only) found a polysemantic attention head, and a head which used the causal attention mask to re-derive positional information. I would love to have some similar case studies of meaningful neurons!

### Empirically Testing Toy Models of Superposition

_The sections of my mech interp explainer on [superposition](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=3br1psLRIjQCOv2T4RN3V6F2) and on the [toy models of superposition](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=EuO4CLwSIzX7AEZA1ZOsnwwF) paper may be useful references_

I'm particularly excited about using Othello-GPT to test and validate some of the predictions of [Toy Models of Superposition](https://transformer-circuits.pub/2022/toy_model/index.html) about what we might find in transformers. Empirical data here seems really valuable! Though there are some important ways that the setup of Othello-GPT differs from their toy model. Notably, they study continuous (uniform \[0, 1\]) features, while Othello-GPT's features seem likely to be binary (on or off), as they're discrete and logical functions of the board state and of the previous moves. Binary features seem more representative of language, especially early token-level features like bigrams and multi-token words, and are also easier to put into superposition, because you don't need to distinguish low values of the correct feature from high values of the incorrect feature

A broader point is whether we expect Othello-GPT to use superposition at all? Their model has more features to represent than dimensions, and so _needs_ to use superposition to pack things in. It's not obvious to me how many features Othello-GPT wants to represent, and how this compares to the number of dimensions - my _guess_ is that it still needs to use superposition, but it's not clear. Some considerations:

*   There's actually a lot of very specific features it might want to learn - eg in the board state -> output logit parts there seems to be a neuron representing C0==BLANK & D1==THEIR'S & E2==MINE, ie can I place a counter in C0 such that it flanks exactly one counter on the diagonal line to the down and right - if this kind of thing is useful, it suggests the model is dealing with a large combinatorial explosion of cases for the many, many similar configurations!
    *   Further, computing the board state from the moves also involves a lot of messy cases, eg dealing with the many times and directions a piece can be flipped and combining this all into a coherent story.
        *   Reminder: Transformers are not recurrent - it can't compute the board state at move n from the state at move n-1, it needs to compute the state at every move _simultaneously_ with just a few layers of attention to move partial computation forwards. This is actually really hard, and it's not obvious to me how you'd implement this in a transformer!
*   There are two different kinds of superposition, residual stream superposition and neuron superposition (ie having more features than dimensions in the residual stream vs in the MLP hidden layer).
    *   The residual stream has 512 dimensions, but there's 8 layers of 2048 neurons each (plus attention heads) - unless many neurons do nothing or are highly redundant, it seems very likely that there's residual stream superposition!
        *   Though note that it's plausible it just has way fewer than 2048 features worth computing, and is massively over-parametrised. I'm not sure what to think here!
        *   The board state alone consumes 25% of the dimensions, if each feature gets a dedicated dimension, and I expect there's probably a bunch of other features worth computing and keeping around?

Concrete questions I'd want to test here - note that the use of dropout may obfuscate these questions (by incentivising redundancy and backup circuits), and this may be best answered in a model without dropout. These also may be best answered in a smaller model with fewer layers and a narrower residual stream, and so with a stronger incentive for superposition!:

*   Do important features get dedicated dimensions in the residual stream? (ie, orthogonal to all other features)
    *   Guesses for important features - whether black or white is playing, the board state, especially features which say which center cells have my colour vs their's.
*   Conversely, can we find evidence that there _is_ overlap between features in the residual stream?
    *   This is surprisingly thorny, since you need to distinguish this kind of genuine interference vs intentional overlapping, eg from the source of the first feature actually wanting to contribute a bit to feature two as well.
*   Do the important neurons seem monosemantic?
    *   Important could mean many things eg high effect when patching, high average activation or standard deviation of activation, high cost when ablated, or any other range of measurements, high gradient or gradient x activation
    *   My workflow would be to use the probe and unembed to interpret neuron weights, max activating dataset examples to help form a hypothesis, and then use a spectrum plot to properly analyse it (discussed more below).
*   Do we get seemingly unrelated features sharing a neuron? The paper predicts superposition is more likely when there are two uncorrelated or anti-correlated features, because then the model doesn't need to track the simultaneous interference of _both_ being there at once.
*   Can we find examples of a feature being computed that _needs_ more than one neuron? Analogous to how eg [modular addition uses ReLUs to multiply two numbers together](https://arxiv.org/pdf/2301.05217.pdf), which takes at least three to do properly. This is a bit of a long shot, since I _think_ any kind of discrete, Boolean operation can probably be done with a single GELU, but I'd love to be proven wrong!
*   Do features actually seem neuron aligned at all?
    *   If we find features in superposition, do they tend to still be sparse (eg linear combinations of 5 ish neurons) or diffuse (no noticable alignment with the neuron basis)
*   Can we find any evidence of spontaneous sorting of superposed features into geometric configurations? (A la the toy models paper)
*   Can you construct any adversarial examples using evidence from the observed polysemanticity?
*   Can you find any circuits used to deal with interference superposition? Or any motifs, like [the asymmetric inhibition motif](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=ZsydEQzRb1l2wuBzqHu9_gQF)?

### Preliminary Results On Neuron Interpretability

_Note that this section has some overlap with results discussed in [my research process](/posts/TAz44Lb9n9yf52pv8/othello-gpt-reflections-on-the-research-process#The_Research_Process)_

In addition to the results above [using the probe to interpret neuron weights](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#Preliminary_Results_On_Modular_Circuits), an obvious place to start is **max activating dataset examples** - run the model over a bunch of games and see what moves the neuron activates the most on. This is actually a fair bit harder to interpret than language, since "what are the connections between these sequences of moves" isn't obvious. I got the most traction from studying board state - in particular, the average number of times each cell is non-empty, and the average number of times a cell is mine vs their's. Here's a plot of the latter for neuron L5N1393 that seems immediately interpretable - D1 is always their's, E2 is always mine! (across 50 games, so 3000 moves) I sometimes get similar results with other layer 5 and layer 6 neurons, though I haven't looked systematically.

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/d8babgvrgumkbgxixl4i" width="600">

Looking at the fraction of the time a cell is blank or not seems to give pretty interesting results for layer 3 and layer 4 neurons.

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/nmxzr2zsjNtjaHh7x/qozdjzolfpembkvjn9iy" width="700">

I expect you can stretch max activating dataset examples further by taking into account more things about the moves - what time in the game they happened, which cells are flipped this turn (and how many times in total!), which cell was played, etc.

My guess from this and probe based analysis earlier was that neuron L5N1393 monosemantically represented the diagonal line configuration C0==BLANK & D1==THEIR'S & E2==MINE. This makes sense as a useful configuration since it says that C0 is a legal move, because it and E2 flank D1! But this seems inconsistent with the direct logit attribution of the neuron (ie the output vector of the neuron projected by the unembed onto the output logits), which seems to boost C0 a lot but also D1 a bit - which seems wildly inconsistent with it firing on D1 being their colour (and thus not a legal place to play!)

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/fsa4iovike7xuoz6tirm" width="500">

These techniques can all be misleading - max activating dataset examples [can cause interpretability illusions](https://arxiv.org/abs/2104.07143), direct logit attribution can fail for neurons that mostly indirectly affect logits, and probes can fail to interpret neurons that mostly read out unrelated features. One of the more robust tools for checking what a neuron means is a **spectrum plot** - if we think a neuron represents some feature, we plot a histogram of the "full spectrum" of the neuron's activations by just taking the neuron activation on a ton of data, and plotting a histogram grouped by whether the feature is present or not (used in [curve detectors](https://distill.pub/2020/circuits/curve-detectors) and [multimodal neurons](https://distill.pub/2021/multimodal-neurons/)). If a neuron is monosemantic, this should fairly cleanly separate into True being high and False being low!

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/ip8scaqsrftsd4qizbga" width="800">

Note that the y axis is percent (ie it's normalised by group size so both True and False's histograms add up to 100 in total, though True is far more spread out so it doesn't look it. This is hard to read, so here it is on a log scale (different to read in a different way!).

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/kgpl9w2vze8we6fmblqk" width="800">

These plots are somewhat hard to interpret, but my impression is that this neuron is plausibly monosemantic-ish, but with a more refined feature - basically all of the high activations have the diagonal line hypothesised, but this is necesssary _not_ sufficient - there's a bunch of negative activations with the line as well! Plausibly it's still monosemantic but there's some extra detail I'm missing, I'm not sure! My next steps would be to refine the hypothesis by inspecting the most positive and most negative True examples, and if I can get a cleaner histogram to then try some causal interventions (eg mean ablating the neuron and seeing if it has the effect my hypothesis would predict). I'd love to see someone finish this analysis, or do a similar deep dive on some other neurons!

Spectrum plots are a pain to make in general, because they require automated feature detectors to do properly (though you can do a janky version by manually inspecting randomly sampled examples, eg a few examples from each decile). One reason I'm excited about neuron interpretability in Othello-GPT is that it's really easy to write automated tests for neurons and thus get spectrum plots, and thus to really investigate monosemanticity! If we want to be able to make real and robust claims to have identified circuits involving neurons or to have mechanistically reverse-engineered a neurons, I want to better understand whether we can claim the neuron is genuinely only used for a single purpose (with noise) or is also used more weakly to represent other features. And a concrete prediction of the toy models framework is that there _should_ be some genuinely monosemantic neurons for the most important features.

That said, showing genuine monosemanticity is hard and spectrum plots are limited. Spectrum plots will still fall down for superposition with very rare features - these can be falsely dismissed as just noise, or just never occur in the games studied! And it's hard to know where to precisely draw the line for "is monosemantic" - it seems unreasonable to say that the smallest True activation must be larger than the largest False one! To me the difference is whether the differences genuinely contribute to the model having low loss, vs on average contributing nothing. I think questions around eg how best to interpret these plots are an example of the kind of practical knowledge I want to get from practicing neuron interpretability!

#### Case Study: Neurons and Probes are Confusing

As a case study in how this can be confusing, here's an earlier draft graph for the section on finding modular circuits - looking at the output weights of top layer 4 neurons (by std) in the blank probe basis. It initially seems like these are all neurons dedicated to computing that a single cell is blank. And I initially got excited and thought this made a great graph for the post! But on reflection this is weird and surprising (exercise: think through why before you read on)

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/yczdmmagc3g8wjz3dvay" width="800">

I argue that this is weird, because figuring out whether a cell is blank should be pretty easy - a cell can never become non-empty, so a cell is blank if and only if it has never been played. This can probably be done in a single attention layer, and the hard part of the world model is computing which cells are mine vs their's. So what's up with this?

It turns out that what's actually going on is that the blank probe is highly correlated with the unembed (the linear map from the final residual to the logits). A cell can be legal _only if_ it is blank, if a cell has a high logit at the end of the model, then it's probably blank. But our probe was computed after layer 6, when there's a lot of extraneous information that probably obscures the blankness information - probably, the probe also learned that if there's going to be a high logit for a cell then that cell is definitely blank, and so the blank directions are partially aligned with the unembed directions. Though on another interpretation, `is_blank` and the unembed are _intentionally_ aligned, because the model knows there's a causal link and so uses the `is_blank` subspace to also contribute to the relevant unembed.

And we see that the alignment with the unembed is even higher! (Around cosine sim of 0.8 to 0.9)

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/qgK7smTvJ4DB8rZ6h/jckacomrew1ju1utpetf" width="800">

## A Transformer Circuit Laboratory

My final category is just the meta level point that I'm confused in many ways about the right **conceptual frameworks** when thinking about transformer circuits, and think that there's a lot of ways we could make progress here! Just as Othello-GPT helped provide notable evidence for the hypothesis that models form linear representations of features, I hope it can help clarify some of these - by concretely understanding what happens inside of it, we can make more informed guesses about transformers in general. Here's a rough brainstorm of weird hypotheses and confusions about what we might find inside transformers - I expect that sufficient investigation of Othello-GPT will shed light on many of them!

Since Othello-GPT is an imperfect proxy for LLMs, it's worth reflecting on what evidence here looks like. I'm most excited about Othello-GPT providing "existence proofs" for mysterious phenomena like memory management: case studies of specific phenomena, making it seem more likely that they arise in real language models. Proofs that something was not used/needed are great, but need to be comprehensive enough to overcome the null hypothesis of "this was/wasn't there but we didn't look hard enough", which is a high bar!

*   Does it do **memory management** in the residual stream? Eg overwriting old features when they're no longer needed. I'd start by looking for neurons with high negative cosine sim between their input and output vectors, ie which basically erase some direction.
    *   One hypothesis is that it implicitly does memory management by increasing the residual stream norm over time - LayerNorm scales it to have fixed norm, so this suppresses earlier features. If this is true, we might instead observe **signal boosting** - key features get systematically boosted over time (eg whether we're playing black or white)
    *   This might come up with cells that flip many times during previous moves - maybe the model changes its guess for the cell's colour back and forth several times as it computes more flips? Do each of these write to the probe direction and overwrite the previous one, or is it something fancier?
*   Do heads and neurons seem like the right units of analysis of the model? Vs eg entire layers, superposition-y linear combinations of neurons/heads, subsets of heads, etc.
*   Do components (heads and neurons) tend to form **tightly integrated circuits** where they strongly compose with just a few other components to form a coherent circuit, or tend to be **modular**, where each component does something coherent in isolation and composes with many other components.
    *   For example, an induction head could be either tightly integrated (the previous token head is highly coupled to the induction head and not used by anything else, and just communicates an encoded message about the previous token directly to the induction head) or could form two separate modules, where the previous token head's output writes to a "what was in the previous position" subspace that many heads (including the induction head!) read from
        *   My guess is the latter, but I don't think anyone's checked! Most working finding concrete circuits seems to focus on patching style investigations on a narrow distribution, rather than broadly checking behaviour on diverse inputs.
    *   On a given input, can we clearly detect which components are composing? Is _this_ sparse?
*   When two components (eg two heads or a head and a neuron) compose with each other, do they tend to write to some shared subspace that many other components read and write from, or is there some specific encod
    *   Do components form modules vs integrated circuits vs etc.
*   Can we find examples of head polysemanticity (a head doing different things in different contexts) or head redundancy (multiple heads doing seemingly the same thing).
    *   Do we see backup heads? That is, heads that compensate for an earlier head when that head is ablated. This model was trained with attention dropout, so I expect they do!
        *   Do these backup heads do anything when _not_ acting as backups?
        *   Can we understand mechanistically how the backup behaviour is implemented?
        *   Are there backup backup heads?
    *   Can we interpret the heads at all? I found this pretty hard, but there must be something legible here!
    *   If we find head redundancy, can we distinguish between head superposition (there's a single "effective head" that consists of a linear combination of these )
    *   Can we find heads which seem to have an attention pattern doing a single thing, but whose OV circuit is used to convey a bunch of different information, read by different downstream circuits
    *   Can we find heads which have very similar attention patterns (ie QK circuits) whose OV circuits add together to simulate a single head with an OV circuit of twice the rank?
*   Is LayerNorm ever used as a meaningful non-linearity (ie, the scale factor differs between tokens in a way that does useful computation), or basically constant? Eg, can you linearly replace it?
    *   Are there emergent features in the residual stream? (ie dimensions in the standard basis that are much bigger than the rest). Do these disproportionately affect LayerNorm?
*   The model has clearly learned some redundancy (because it was trained with dropout, but also likely would learn some without any dropout). How is this represented mechanistically?
    *   Is it about having backup circuits that takeover when the first thing is ablated? Multiple directions for the same feature? Etc.
*   Can you find more evidence for or against the hypothesis that features are represented linearly?
    *   If so, do these get represented orthogonally?
*   Ambitiously, do we have a shot at figuring out _everything_ that the model is doing? Does it seem remotely possible to fully-reverse engineer it?
    *   Is there a long tail of fuzzy, half-formed features that aren't clean enough to interpret, but slightly damage loss if ablated? Are there neurons that just do nothing either way?
    *   Some ambitious plans for interpretability for alignment involve aiming for [enumerative safety](https://transformer-circuits.pub/2022/toy_model/index.html#strategic-safety), the idea that we might be able to enumerate _all_ features in a model and inspect this for features related to dangerous capabilities or intentions. Seeing whether this is remotely possible for Othello-GPT may be a decent test run.
*   Do the residual stream or internal head vectors have a privileged basis? Both with [statistical tests like kurtosis](https://transformer-circuits.pub/2023/privileged-basis/index.html), and in terms of whether you can actually interp directions in the standard basis?
*   Do transformers behave like [ensembles of shallow paths](https://arxiv.org/abs/1605.06431)? Where each meaningful circuit tends to only involve a few of the 16 sublayers, and makes heavy use of the residual stream (rather than 16 serial steps of computation).
    *   Prior circuits work and techniques like [the logit lens](/posts/AcKRB8wDpdaN6v6ru/interpreting-gpt-the-logit-lens) seems to heavily imply this, but it would be good to get more data!
    *   A related hypothesis - when a circuit involves several components (eg a feature is computed by several neurons in tandem) are these always in the same layer? One of my fears is that superposition gives rise to features that are eg linear combinations of 5 neurons, but that these are spread across adjacent layers!

## Where to start?

If you've read this far, hopefully I've convinced you there are interesting directions here that could be worth working on! The next natural question is, where to start? Some thoughts:

*   Read the [original paper carefully](https://arxiv.org/pdf/2210.13382.pdf)
*   If you're new to mech interp, check out [my getting started guide](https://neelnanda.io/getting-started).
    *   I particularly recommend getting your head around how a transformer works, and being familiar with linear algebra
*   Use [my accompanying notebook](https://neelnanda.io/othello-notebook) as a starting point which demonstrates many of the core techniques
    *   I highly recommend using [my TransformerLens library](https://github.com/neelnanda-io/TransformerLens) for this, I designed it to enable this kind of research
    *   [Check out the underlying codebase](https://github.com/likenneth/othello_world) (made by the original authors, thanks to Kenneth Li for the code and for letting me make additions!)
*   My [concrete open problems](https://neelnanda.io/concrete-open-problems) sequence has a bunch of tips on doing good mech interp research, especially in the posts on circuits in toy language models, on neuron interpretability, and on superposition.
*   Read through [my notes on my research process](/posts/TAz44Lb9n9yf52pv8/othello-gpt-reflections-on-the-research-process#The_Research_Process) to get a sense of what making progress on this kind of work looks like, and in particular the decisions I made and why.

### Concrete starter projects

I'll now try to detail some concrete open problems that I think could be good places to start. Note that these are just preliminary suggestions - the above sections outline my underlying philosophy of which questions I'm excited about and a bunch of scattered thoughts about how to make progress on them. If there's a direction you personally feel excited about, you should just jump in.

Ideas for gentle starter projects (Note that I have not actually tried these - I expect them to be easy, but I expect at least one is actually cursed! If you get super stuck, just move on):

*   How does the model decide that the cell for the current move is not blank?
    *   What's the natural way for a transformer to implement this? (Hint: Do you need information about previous moves to answer this?)
    *   At which layer has the model figured this out?
    *   Try patching between two possibilities for the current move (with the same previous game) and look at what's going on
*   Pick a specific cell (eg B3). How does the model compute that it's blank?
    *   I'd start by studying the model on a few specific moves. At which layer does the model conclude that it's blank? Does this come from any specific head or neuron?
    *   Conceptually, a cell is not blank if and only if it was played as a previous move - how could a transformer detect this? (Hint: A single attention head per cell would work)
*   Take a game where a center cell gets flipped many times. Look at what colour the model thinks that cell is, after each layer and move. What patterns can you see? Can you form any guesses about what's going on? (This is a high-level project - the goal is to form hypotheses, not to reach clear answers)
*   Take the `is_my_colour` direction for a specific cell (eg D7) and look for neurons whose input weight has high cosine similarity with this. Look at this neuron's cosine sim with every other probe direction, and form a guess about what it's doing (if it's a mess then try another neuron/cell). Example guesses might be
    *   Then look at the max activating dataset examples (eg the top 10 over 50 games) and check if your guess worked!
    *   Extension: Plot a [spectrum plot](/posts/qgK7smTvJ4DB8rZ6h/othello-gpt-future-work-i-am-excited-about#Preliminary_Results_On_Neuron_Interpretability) and check how monosemantic it actually is
*   Repeat the above for the `is_blank` direction.
*   Take the average of the even minus the average of the odd positional embeddings to get an "I am playing white" direction. Does this seem to get its own dedicated dimension, or is it in superposition?
    *   A hard part about answering this question is distinguishing there being non-orthogonal features, vs other components doing memory management and eg systematically signal boosting the "I am playing white" direction so it's a constant fraction of the residual stream. Memory management should act approximately the same between games, while other features won't.

### Cleaning Up

This was (deliberately!) a pretty rushed and shallow investigation, and I cut a bunch of corners. There's some basic cleaning up I would do if I wanted to turn this into a real paper or build a larger project, and this might be a good place to start!

*   Training a better probe: I cut a lot of corners in training this probe... Some ideas:
    *   Train it on both black and white moves! (to predict my vs their's, so flip the state every other move)
    *   I cut out the first and last 5 moves - does this actually help/matter? Check how well the current probe works on early and late moves.
    *   The state of different cells will be correlated (eg a corner can only be filled if a neighbouring cell is filled), so the probes may be non-orthogonal for boring reasons. Does it help to constrain them to be orthogonal?
    *   What's the right layer to train a probe on?
    *   The probe is 3 vectors (three-way logistic regression), but I want a `is_blank_vs_filled` and `is_mine_vs_theirs_conditional_on_not_being_blank` direction - what's the most principled way of doing this?
*   Rigorously testing interventions: I'm pretty convinced that intervening the probe does _something_, but
    *   Currently I take the _current_ coordinate with respect to the probe direction, negate that, and then scale. Plausibly, this is dumb and the magnitude of the original coordinate doesn't matter, and I should instead replace it with a constant magnitude. The place I'd start is to just plot a histogram of the coordinates in the probe directions
    *   Replicating the paper's analysis of whether their intervention works (their natural and unnatural benchmark)
*   Re-train the model: The model was trained with attention and residual dropout - this is not representative of modern LLMs, and incentivises messy and redundant representations and backup circuits, I expect that training a new model from scratch with no dropout will make your life much easier. _(Note that someone is currently working on this)_
    *   The current model is 8 layers with a residual stream of width 512. I speculate this is actually much bigger than it needs to be, and things might be cleaner with fewer layers and a wider stream, a narrower stream, or both.

## Suggested paper replications

### [Inside the mind of a superhuman Go model: How does Leela Zero read ladders?](https://www.lesswrong.com/posts/FF8i6SLfKb4g7C4EL/inside-the-mind-of-a-superhuman-go-model-how-does-leela-zero-2)

The authors do some interpretability on Leela Zero, a superhuman Go model. With a technique similar to the logit lens, they find that the residual structure of Leela Zero induces a preferred basis throughout network, giving rise to persistent, interpretable channels. By directly analyzing the weights of the policy and value heads, they found that the model stores information related to the probability of the pass move along the top edge of the board, and those related to the board value in checkerboard patterns. The authors also took a deep dive into a specific Go technique, the ladder, and identified a very small subset of model components that are causally responsible for the model's judgement of ladders.

Adapting interpretability techniques to a model like this might be quite challenging (Leela Zero is a deep CNN, based more on AlphaGo's architecture than on any transformer model). However, replicating some of these results might be a good challenge!

This might be a good replication for you if:

- You enjoyed the exercises in this section
- You enjoyed thinking about game mechanisms of OthelloGPT, and taking deep dives into specific mechanisms used by the model
- You're excited by the idea of applying standard interpretability techniques (e.g. logit lens and neuron analysis) to different architectures
- You know how to play Go (weakly recommended, not required!)

### [Chess-GPT's Internal World Model](https://adamkarvonen.github.io/machine_learning/2024/01/03/chess-world-models.html)

In this work, Adam Karvonen extends Neel Nanda's OthelloGPT methodology, applying it to a chess model which he trained. THe model is a 50 million parameter GPT, trained on 5 million games of chess, and it learned to play at ~1300 Elo in one day on 4 RTX 3090 GPU.

The analysis in the linked post provides some brief dives into the model, as well as several suggested future directions. Can you replicate the results in the post, and investigate some of the suggested future directions?

This might be a good replication for you if:

- You enjoyed the exercises in this section
- You enjoyed thinking about game mechanisms of OthelloGPT, and taking deep dives into specific mechanisms used by the model
- You're more comfortable with the rules of chess than with Go
- You'd like a project on the slightly easier end (at least relative to the LeelaZero work, since this will be a lot closer to the exercises you've already done)